In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import scipy.io
import seaborn as sns
import os
import glob
import mne
# import copy
# import logging
# import mne_connectivity
# import networkx as nx
# from statsmodels.tsa.stattools import grangercausalitytests
# from pyinform.mutualinfo import mutual_info
# from statsmodels.tsa.vector_ar.var_model import VAR
from scipy.signal import coherence
# from scipy.signal import csd, welch
# from matplotlib import cm
from scipy.signal import hilbert
# from matplotlib.colors import LogNorm
from mne_connectivity import spectral_connectivity_epochs

# Coherence Matrix Depression Dataset

## Alpha Band

In [ ]:
# electrode_name_mapping = {
#     'EEG039': 'FT7',
#     'EEG045': 'T7',
#     'EEG050': 'TP7',
#     'EEG101': 'TP8',
#     'EEG108': 'T8',
#     'EEG115': 'FT8',
#     'EEG024': 'F3',
#     'EEG124': 'F4'
# }

def find_fif_files(directory):
    return [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.fif')]

channel_groups = [
    [
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\filter_channel_FT7_depression\alpha_band_FT7_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\filter_channel_T7_depression\alpha_band_T7_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\filter_channel_TP7_depression\alpha_band_TP7_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\filter_channel_TP8_depression\alpha_band_TP8_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\filter_channel_T8_depression\alpha_band_T8_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\filter_channel_FT8_depression\alpha_band_FT8_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\filter_channel_F3_depression\alpha_band_F3_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\filter_channel_F4_depression\alpha_band_F4_depression"
    ]
]

for channel_group in channel_groups:
    all_epochs_data = []  # This will hold the data for all epochs across all channels
    ch_names = ['FT7', 'T7', 'TP7', 'TP8', 'T8', 'FT8', 'F3', 'F4']
    info = None  # This will be used to store the info structure for creating combined epochs later
    for channel_dir in channel_group:
        channel_epochs_data = []  # This will hold data for all epochs for the current channel
        try:
            file_paths = find_fif_files(channel_dir)
            if not file_paths:
                print(f"No .fif files found in {channel_dir}. Skipping this directory.")
                continue
            for file_path in file_paths:
                try:
                    epochs = mne.read_epochs(file_path, preload=True)
                    print(f"{file_path} contains {len(epochs)} epochs")
                    if info is None:
                        info = epochs.info  # We only need to set this once
                    channel_epochs_data.append(epochs.get_data())  # Append data for these epochs
                except Exception as e:
                    print(f"Failed to read epochs from {file_path}. Error: {e}")
                    continue
        except Exception as e:
            print(f"Error processing directory {channel_dir}: {e}")
            continue
        if channel_epochs_data:
            # Concatenate all epoch data for the current channel
            channel_basename = os.path.basename(channel_dir)
            total_epochs = sum(len(e) for e in channel_epochs_data)
            print(f"Channel {channel_basename} combined total epochs: {total_epochs}")
            channel_combined_data = np.concatenate(channel_epochs_data, axis=0)
            all_epochs_data.append(channel_combined_data)

    if all_epochs_data:
        # Combine data from all channels, adjusting axis as necessary
        data_combined = np.concatenate(all_epochs_data, axis=1)  # Adjust axis based on your data structure
        print(f"Total combined epochs across all channels: {data_combined.shape[0]}")
        print(f"Diagnostic: Combined data shape: {data_combined.shape}")
        
    else:
        print("No epochs were loaded. Please check file paths and formats.")
        continue

    # Now, create the combined epochs structure
    if info is not None and data_combined.size > 0:
        # mapped_ch_names = [electrode_name_mapping.get(ch_name, ch_name) for ch_name in ch_names]
        # Update info to match the combined data
        info = mne.create_info(ch_names=ch_names, sfreq=info['sfreq'], ch_types=['eeg'] * len(ch_names))
        # Create a dummy events array and event_id dict, as they are required for creating EpochsArray
        events = np.array([[i, 0, 1] for i in range(data_combined.shape[0])])
        event_id = {'dummy_event': 1}
        epochs_combined = mne.EpochsArray(data_combined, info, events=events, event_id=event_id)
        print(f"Starting connectivity analysis on combined data...")
        processed_epochs_count = 0
        save_path = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\coherence_visualization_depression\alpha_band_coherence_depression"
        save_csv_path = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\csv_data_depression\alpha_band_csv_depression"
        for epoch_idx in range(len(epochs_combined)):
            # Here you can proceed with your connectivity analysis on `epochs_combined`
            con = spectral_connectivity_epochs(
                epochs_combined[epoch_idx:epoch_idx+1], method='coh', mode='multitaper', sfreq=epochs_combined.info['sfreq'],
                fmin=8, fmax=12, faverage=True, mt_adaptive=True, n_jobs=1
            )
            con_matrix = con.get_data(output='dense')[..., 0]
            df_con = pd.DataFrame(con_matrix, index=ch_names, columns=ch_names)
            processed_epochs_count += 1
            
            csv_file_name = f"coherence_matrix_Depression_alphaBand_epoch_{epoch_idx + 1}.csv"
            csv_full_path = os.path.join(save_csv_path, csv_file_name)
            df_con.to_csv(csv_full_path)
            
            plt.figure(figsize=(10, 8))
            sns.heatmap(df_con, annot=True, cmap='viridis', fmt=".2f")
            plt.title('Coherence Matrix for Alpha Band Depression')
            plt.ylabel('Channels')
            plt.xlabel('Channels')
            file_name = f"coherence_matrix_Depression_alphaBand_epoch_{epoch_idx + 1}.png"
            full_path = os.path.join(save_path, file_name)
            plt.savefig(full_path)
            plt.close()
            print(f"Processed and visualized {processed_epochs_count} epochs so far.")
            print(f"Total epochs processed for visualization: {processed_epochs_count}")
        else:
            print("No combined data available for creating epochs.")


## Alpha Band (dataset real)

In [ ]:
# electrode_name_mapping = {
#     'EEG039': 'FT7',
#     'EEG045': 'T7',
#     'EEG050': 'TP7',
#     'EEG101': 'TP8',
#     'EEG108': 'T8',
#     'EEG115': 'FT8',
#     'EEG024': 'F3',
#     'EEG124': 'F4'
# }

def find_edf_files(directory):
    return [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.edf')]

channel_groups = [
    [
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\filter_channel_C3-R_depression\alpha_band_C3-R_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\filter_channel_C4-R_depression\alpha_band_C4-R_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\filter_channel_FP1-R_depression\alpha_band_FP1-R_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\filter_channel_FP2-R_depression\alpha_band_FP2-R_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\filter_channel_O1-R_depression\alpha_band_O1-R_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\filter_channel_O2-R_depression\alpha_band_O2-R_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\filter_channel_T3-R_depression\alpha_band_T3-R_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\filter_channel_T4-R_depression\alpha_band_T4-R_depression"
    ]
]

for channel_group in channel_groups:
    all_epochs_data = []  # This will hold the data for all epochs across all channels
    ch_names = ['EEG FP1-R', 'EEG FP2-R', 'EEG C3-R', 'EEG C4-R', 'EEG O1-R', 'EEG O2-R', 'EEG T3-R', 'EEG T4-R']
    info = None  # This will be used to store the info structure for creating combined epochs later
    for channel_dir in channel_group:
        channel_epochs_data = []  # This will hold data for all epochs for the current channel
        try:
            file_paths = find_edf_files(channel_dir)
            if not file_paths:
                print(f"No .edf files found in {channel_dir}. Skipping this directory.")
                continue
            for file_path in file_paths:
                try:
                    epochs = mne.read_epochs(file_path, preload=True)
                    print(f"{file_path} contains {len(epochs)} epochs")
                    if info is None:
                        info = epochs.info  # We only need to set this once
                    channel_epochs_data.append(epochs.get_data())  # Append data for these epochs
                except Exception as e:
                    print(f"Failed to read epochs from {file_path}. Error: {e}")
                    continue
        except Exception as e:
            print(f"Error processing directory {channel_dir}: {e}")
            continue
        if channel_epochs_data:
            # Concatenate all epoch data for the current channel
            channel_basename = os.path.basename(channel_dir)
            total_epochs = sum(len(e) for e in channel_epochs_data)
            print(f"Channel {channel_basename} combined total epochs: {total_epochs}")
            channel_combined_data = np.concatenate(channel_epochs_data, axis=0)
            all_epochs_data.append(channel_combined_data)

    if all_epochs_data:
        # Combine data from all channels, adjusting axis as necessary
        data_combined = np.concatenate(all_epochs_data, axis=1)  # Adjust axis based on your data structure
        print(f"Total combined epochs across all channels: {data_combined.shape[0]}")
        print(f"Diagnostic: Combined data shape: {data_combined.shape}")
        
    else:
        print("No epochs were loaded. Please check file paths and formats.")
        continue

    # Now, create the combined epochs structure
    if info is not None and data_combined.size > 0:
        # mapped_ch_names = [electrode_name_mapping.get(ch_name, ch_name) for ch_name in ch_names]
        # Update info to match the combined data
        info = mne.create_info(ch_names=ch_names, sfreq=info['sfreq'], ch_types=['eeg'] * len(ch_names))
        # Create a dummy events array and event_id dict, as they are required for creating EpochsArray
        events = np.array([[i, 0, 1] for i in range(data_combined.shape[0])])
        event_id = {'dummy_event': 1}
        epochs_combined = mne.EpochsArray(data_combined, info, events=events, event_id=event_id)
        print(f"Starting connectivity analysis on combined data...")
        processed_epochs_count = 0
        save_path = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\coherence_visualization_depression\alpha_band_coherence_depression"
        save_csv_path = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\csv_data_coherence_depression\alpha_band_csv_depression"
        for epoch_idx in range(len(epochs_combined)):
            # Here you can proceed with your connectivity analysis on `epochs_combined`
            con = spectral_connectivity_epochs(
                epochs_combined[epoch_idx:epoch_idx+1], method='coh', mode='multitaper', sfreq=epochs_combined.info['sfreq'],
                fmin=8, fmax=12, faverage=True, mt_adaptive=True, n_jobs=1
            )
            con_matrix = con.get_data(output='dense')[..., 0]
            df_con = pd.DataFrame(con_matrix, index=ch_names, columns=ch_names)
            processed_epochs_count += 1
            
            csv_file_name = f"coherence_matrix_Depression_alphaBand_epoch_{epoch_idx + 1}.csv"
            csv_full_path = os.path.join(save_csv_path, csv_file_name)
            df_con.to_csv(csv_full_path)
            
            plt.figure(figsize=(10, 8))
            sns.heatmap(df_con, annot=True, cmap='viridis', fmt=".2f")
            plt.title('Coherence Matrix for Alpha Band Depression')
            plt.ylabel('Channels')
            plt.xlabel('Channels')
            file_name = f"coherence_matrix_Depression_alphaBand_epoch_{epoch_idx + 1}.png"
            full_path = os.path.join(save_path, file_name)
            plt.savefig(full_path)
            plt.close()
            print(f"Processed and visualized {processed_epochs_count} epochs so far.")
            print(f"Total epochs processed for visualization: {processed_epochs_count}")
        else:
            print("No combined data available for creating epochs.")


## Beta Band

In [ ]:
# electrode_name_mapping = {
#     'EEG039': 'FT7',
#     'EEG045': 'T7',
#     'EEG050': 'TP7',
#     'EEG101': 'TP8',
#     'EEG108': 'T8',
#     'EEG115': 'FT8',
#     'EEG024': 'F3',
#     'EEG124': 'F4'
# }

def find_fif_files(directory):
    return [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.fif')]

channel_groups = [
    [
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\filter_channel_FT7_depression\beta_band_FT7_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\filter_channel_T7_depression\beta_band_T7_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\filter_channel_TP7_depression\beta_band_TP7_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\filter_channel_TP8_depression\beta_band_TP8_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\filter_channel_T8_depression\beta_band_T8_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\filter_channel_FT8_depression\beta_band_FT8_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\filter_channel_F3_depression\beta_band_F3_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\filter_channel_F4_depression\beta_band_F4_depression"
    ]
]

for channel_group in channel_groups:
    all_epochs_data = []  # This will hold the data for all epochs across all channels
    ch_names = ['FT7', 'T7', 'TP7', 'TP8', 'T8', 'FT8', 'F3', 'F4']
    info = None  # This will be used to store the info structure for creating combined epochs later
    for channel_dir in channel_group:
        channel_epochs_data = []  # This will hold data for all epochs for the current channel
        try:
            file_paths = find_fif_files(channel_dir)
            if not file_paths:
                print(f"No .fif files found in {channel_dir}. Skipping this directory.")
                continue
            for file_path in file_paths:
                try:
                    epochs = mne.read_epochs(file_path, preload=True)
                    print(f"{file_path} contains {len(epochs)} epochs")
                    if info is None:
                        info = epochs.info  # We only need to set this once
                    channel_epochs_data.append(epochs.get_data())  # Append data for these epochs
                except Exception as e:
                    print(f"Failed to read epochs from {file_path}. Error: {e}")
                    continue
        except Exception as e:
            print(f"Error processing directory {channel_dir}: {e}")
            continue
        if channel_epochs_data:
            # Concatenate all epoch data for the current channel
            channel_basename = os.path.basename(channel_dir)
            total_epochs = sum(len(e) for e in channel_epochs_data)
            print(f"Channel {channel_basename} combined total epochs: {total_epochs}")
            channel_combined_data = np.concatenate(channel_epochs_data, axis=0)
            all_epochs_data.append(channel_combined_data)

    if all_epochs_data:
        # Combine data from all channels, adjusting axis as necessary
        data_combined = np.concatenate(all_epochs_data, axis=1)  # Adjust axis based on your data structure
        print(f"Total combined epochs across all channels: {data_combined.shape[0]}")
        print(f"Diagnostic: Combined data shape: {data_combined.shape}")
        
    else:
        print("No epochs were loaded. Please check file paths and formats.")
        continue

    # Now, create the combined epochs structure
    if info is not None and data_combined.size > 0:
        # mapped_ch_names = [electrode_name_mapping.get(ch_name, ch_name) for ch_name in ch_names]
        # Update info to match the combined data
        info = mne.create_info(ch_names=ch_names, sfreq=info['sfreq'], ch_types=['eeg'] * len(ch_names))
        # Create a dummy events array and event_id dict, as they are required for creating EpochsArray
        events = np.array([[i, 0, 1] for i in range(data_combined.shape[0])])
        event_id = {'dummy_event': 1}
        epochs_combined = mne.EpochsArray(data_combined, info, events=events, event_id=event_id)
        print(f"Starting connectivity analysis on combined data...")
        processed_epochs_count = 0
        save_path = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\coherence_visualization_depression\beta_band_coherence_depression"
        save_csv_path = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\csv_data_depression\beta_band_csv_depression"
        for epoch_idx in range(len(epochs_combined)):
            # Here you can proceed with your connectivity analysis on `epochs_combined`
            con = spectral_connectivity_epochs(
                epochs_combined[epoch_idx:epoch_idx+1], method='coh', mode='multitaper', sfreq=epochs_combined.info['sfreq'],
                fmin=13, fmax=30, faverage=True, mt_adaptive=True, n_jobs=1
            )
            con_matrix = con.get_data(output='dense')[..., 0]
            df_con = pd.DataFrame(con_matrix, index=ch_names, columns=ch_names)
            processed_epochs_count += 1
            
            csv_file_name = f"coherence_matrix_Depression_betaBand_epoch_{epoch_idx + 1}.csv"
            csv_full_path = os.path.join(save_csv_path, csv_file_name)
            df_con.to_csv(csv_full_path)
            
            plt.figure(figsize=(10, 8))
            sns.heatmap(df_con, annot=True, cmap='viridis', fmt=".2f")
            plt.title('Coherence Matrix for Beta Band Depression')
            plt.ylabel('Channels')
            plt.xlabel('Channels')
            file_name = f"coherence_matrix_Depression_betaBand_epoch_{epoch_idx + 1}.png"
            full_path = os.path.join(save_path, file_name)
            plt.savefig(full_path)
            plt.close()
            print(f"Processed and visualized {processed_epochs_count} epochs so far.")
            print(f"Total epochs processed for visualization: {processed_epochs_count}")
        else:
            print("No combined data available for creating epochs.")


## Beta band (dataset real)

In [ ]:
# electrode_name_mapping = {
#     'EEG039': 'FT7',
#     'EEG045': 'T7',
#     'EEG050': 'TP7',
#     'EEG101': 'TP8',
#     'EEG108': 'T8',
#     'EEG115': 'FT8',
#     'EEG024': 'F3',
#     'EEG124': 'F4'
# }

def find_edf_files(directory):
    return [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.edf')]

channel_groups = [
    [
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\filter_channel_C3-R_depression\beta_band_C3-R_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\filter_channel_C4-R_depression\beta_band_C4-R_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\filter_channel_FP1-R_depression\beta_band_FP1-R_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\filter_channel_FP2-R_depression\beta_band_FP2-R_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\filter_channel_O1-R_depression\beta_band_O1-R_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\filter_channel_O2-R_depression\beta_band_O2-R_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\filter_channel_T3-R_depression\beta_band_T3-R_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\filter_channel_T4-R_depression\beta_band_T4-R_depression"
    ]
]

for channel_group in channel_groups:
    all_epochs_data = []  # This will hold the data for all epochs across all channels
    ch_names = ['EEG FP1-R', 'EEG FP2-R', 'EEG C3-R', 'EEG C4-R', 'EEG O1-R', 'EEG O2-R', 'EEG T3-R', 'EEG T4-R']
    info = None  # This will be used to store the info structure for creating combined epochs later
    for channel_dir in channel_group:
        channel_epochs_data = []  # This will hold data for all epochs for the current channel
        try:
            file_paths = find_edf_files(channel_dir)
            if not file_paths:
                print(f"No .edf files found in {channel_dir}. Skipping this directory.")
                continue
            for file_path in file_paths:
                try:
                    epochs = mne.read_epochs(file_path, preload=True)
                    print(f"{file_path} contains {len(epochs)} epochs")
                    if info is None:
                        info = epochs.info  # We only need to set this once
                    channel_epochs_data.append(epochs.get_data())  # Append data for these epochs
                except Exception as e:
                    print(f"Failed to read epochs from {file_path}. Error: {e}")
                    continue
        except Exception as e:
            print(f"Error processing directory {channel_dir}: {e}")
            continue
        if channel_epochs_data:
            # Concatenate all epoch data for the current channel
            channel_basename = os.path.basename(channel_dir)
            total_epochs = sum(len(e) for e in channel_epochs_data)
            print(f"Channel {channel_basename} combined total epochs: {total_epochs}")
            channel_combined_data = np.concatenate(channel_epochs_data, axis=0)
            all_epochs_data.append(channel_combined_data)

    if all_epochs_data:
        # Combine data from all channels, adjusting axis as necessary
        data_combined = np.concatenate(all_epochs_data, axis=1)  # Adjust axis based on your data structure
        print(f"Total combined epochs across all channels: {data_combined.shape[0]}")
        print(f"Diagnostic: Combined data shape: {data_combined.shape}")
        
    else:
        print("No epochs were loaded. Please check file paths and formats.")
        continue

    # Now, create the combined epochs structure
    if info is not None and data_combined.size > 0:
        # mapped_ch_names = [electrode_name_mapping.get(ch_name, ch_name) for ch_name in ch_names]
        # Update info to match the combined data
        info = mne.create_info(ch_names=ch_names, sfreq=info['sfreq'], ch_types=['eeg'] * len(ch_names))
        # Create a dummy events array and event_id dict, as they are required for creating EpochsArray
        events = np.array([[i, 0, 1] for i in range(data_combined.shape[0])])
        event_id = {'dummy_event': 1}
        epochs_combined = mne.EpochsArray(data_combined, info, events=events, event_id=event_id)
        print(f"Starting connectivity analysis on combined data...")
        processed_epochs_count = 0
        save_path = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\coherence_visualization_depression\beta_band_coherence_depression"
        save_csv_path = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\csv_data_coherence_depression\beta_band_csv_depression"
        for epoch_idx in range(len(epochs_combined)):
            # Here you can proceed with your connectivity analysis on `epochs_combined`
            con = spectral_connectivity_epochs(
                epochs_combined[epoch_idx:epoch_idx+1], method='coh', mode='multitaper', sfreq=epochs_combined.info['sfreq'],
                fmin=13, fmax=30, faverage=True, mt_adaptive=True, n_jobs=1
            )
            con_matrix = con.get_data(output='dense')[..., 0]
            df_con = pd.DataFrame(con_matrix, index=ch_names, columns=ch_names)
            processed_epochs_count += 1
            
            csv_file_name = f"coherence_matrix_Depression_betaBand_epoch_{epoch_idx + 1}.csv"
            csv_full_path = os.path.join(save_csv_path, csv_file_name)
            df_con.to_csv(csv_full_path)
            
            plt.figure(figsize=(10, 8))
            sns.heatmap(df_con, annot=True, cmap='viridis', fmt=".2f")
            plt.title('Coherence Matrix for Beta Band Depression')
            plt.ylabel('Channels')
            plt.xlabel('Channels')
            file_name = f"coherence_matrix_Depression_betaBand_epoch_{epoch_idx + 1}.png"
            full_path = os.path.join(save_path, file_name)
            plt.savefig(full_path)
            plt.close()
            print(f"Processed and visualized {processed_epochs_count} epochs so far.")
            print(f"Total epochs processed for visualization: {processed_epochs_count}")
        else:
            print("No combined data available for creating epochs.")


## Delta Band

In [ ]:
# electrode_name_mapping = {
#     'EEG039': 'FT7',
#     'EEG045': 'T7',
#     'EEG050': 'TP7',
#     'EEG101': 'TP8',
#     'EEG108': 'T8',
#     'EEG115': 'FT8',
#     'EEG024': 'F3',
#     'EEG124': 'F4'
# }

def find_fif_files(directory):
    return [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.fif')]

channel_groups = [
    [
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\filter_channel_FT7_depression\delta_band_FT7_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\filter_channel_T7_depression\delta_band_T7_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\filter_channel_TP7_depression\delta_band_TP7_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\filter_channel_TP8_depression\delta_band_TP8_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\filter_channel_T8_depression\delta_band_T8_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\filter_channel_FT8_depression\delta_band_FT8_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\filter_channel_F3_depression\delta_band_F3_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\filter_channel_F4_depression\delta_band_F4_depression"
    ]
]

for channel_group in channel_groups:
    all_epochs_data = []  # This will hold the data for all epochs across all channels
    ch_names = ['FT7', 'T7', 'TP7', 'TP8', 'T8', 'FT8', 'F3', 'F4']
    info = None  # This will be used to store the info structure for creating combined epochs later
    for channel_dir in channel_group:
        channel_epochs_data = []  # This will hold data for all epochs for the current channel
        try:
            file_paths = find_fif_files(channel_dir)
            if not file_paths:
                print(f"No .fif files found in {channel_dir}. Skipping this directory.")
                continue
            for file_path in file_paths:
                try:
                    epochs = mne.read_epochs(file_path, preload=True)
                    print(f"{file_path} contains {len(epochs)} epochs")
                    if info is None:
                        info = epochs.info  # We only need to set this once
                    channel_epochs_data.append(epochs.get_data())  # Append data for these epochs
                except Exception as e:
                    print(f"Failed to read epochs from {file_path}. Error: {e}")
                    continue
        except Exception as e:
            print(f"Error processing directory {channel_dir}: {e}")
            continue
        if channel_epochs_data:
            # Concatenate all epoch data for the current channel
            channel_basename = os.path.basename(channel_dir)
            total_epochs = sum(len(e) for e in channel_epochs_data)
            print(f"Channel {channel_basename} combined total epochs: {total_epochs}")
            channel_combined_data = np.concatenate(channel_epochs_data, axis=0)
            all_epochs_data.append(channel_combined_data)

    if all_epochs_data:
        # Combine data from all channels, adjusting axis as necessary
        data_combined = np.concatenate(all_epochs_data, axis=1)  # Adjust axis based on your data structure
        print(f"Total combined epochs across all channels: {data_combined.shape[0]}")
        print(f"Diagnostic: Combined data shape: {data_combined.shape}")
        
    else:
        print("No epochs were loaded. Please check file paths and formats.")
        continue

    # Now, create the combined epochs structure
    if info is not None and data_combined.size > 0:
        # mapped_ch_names = [electrode_name_mapping.get(ch_name, ch_name) for ch_name in ch_names]
        # Update info to match the combined data
        info = mne.create_info(ch_names=ch_names, sfreq=info['sfreq'], ch_types=['eeg'] * len(ch_names))
        # Create a dummy events array and event_id dict, as they are required for creating EpochsArray
        events = np.array([[i, 0, 1] for i in range(data_combined.shape[0])])
        event_id = {'dummy_event': 1}
        epochs_combined = mne.EpochsArray(data_combined, info, events=events, event_id=event_id)
        print(f"Starting connectivity analysis on combined data...")
        processed_epochs_count = 0
        save_path = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\coherence_visualization_depression\delta_band_coherence_depression"
        save_csv_path = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\csv_data_depression\delta_band_csv_depression"
        for epoch_idx in range(len(epochs_combined)):
            # Here you can proceed with your connectivity analysis on `epochs_combined`
            con = spectral_connectivity_epochs(
                epochs_combined[epoch_idx:epoch_idx+1], method='coh', mode='multitaper', sfreq=epochs_combined.info['sfreq'],
                fmin=0.5, fmax=4, faverage=True, mt_adaptive=True, n_jobs=1
            )
            con_matrix = con.get_data(output='dense')[..., 0]
            df_con = pd.DataFrame(con_matrix, index=ch_names, columns=ch_names)
            processed_epochs_count += 1
            
            csv_file_name = f"coherence_matrix_Depression_deltaBand_epoch_{epoch_idx + 1}.csv"
            csv_full_path = os.path.join(save_csv_path, csv_file_name)
            df_con.to_csv(csv_full_path)

            plt.figure(figsize=(10, 8))
            sns.heatmap(df_con, annot=True, cmap='viridis', fmt=".2f")
            plt.title('Coherence Matrix for Delta Band Depression')
            plt.ylabel('Channels')
            plt.xlabel('Channels')
            file_name = f"coherence_matrix_Depression_deltaBand_epoch_{epoch_idx + 1}.png"
            full_path = os.path.join(save_path, file_name)
            plt.savefig(full_path)
            plt.close()
            print(f"Processed and visualized {processed_epochs_count} epochs so far.")
            print(f"Total epochs processed for visualization: {processed_epochs_count}")
        else:
            print("No combined data available for creating epochs.")


## Delta band (dataset real)

In [ ]:
# electrode_name_mapping = {
#     'EEG039': 'FT7',
#     'EEG045': 'T7',
#     'EEG050': 'TP7',
#     'EEG101': 'TP8',
#     'EEG108': 'T8',
#     'EEG115': 'FT8',
#     'EEG024': 'F3',
#     'EEG124': 'F4'
# }

def find_edf_files(directory):
    return [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.edf')]

channel_groups = [
    [
       r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\filter_channel_C3-R_depression\delta_band_C3-R_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\filter_channel_C4-R_depression\delta_band_C4-R_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\filter_channel_FP1-R_depression\delta_band_FP1-R_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\filter_channel_FP2-R_depression\delta_band_FP2-R_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\filter_channel_O1-R_depression\delta_band_O1-R_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\filter_channel_O2-R_depression\delta_band_O2-R_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\filter_channel_T3-R_depression\delta_band_T3-R_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\filter_channel_T4-R_depression\delta_band_T4-R_depression"
    ]
]

for channel_group in channel_groups:
    all_epochs_data = []  # This will hold the data for all epochs across all channels
    ch_names = ['EEG FP1-R', 'EEG FP2-R', 'EEG C3-R', 'EEG C4-R', 'EEG O1-R', 'EEG O2-R', 'EEG T3-R', 'EEG T4-R']
    info = None  # This will be used to store the info structure for creating combined epochs later
    for channel_dir in channel_group:
        channel_epochs_data = []  # This will hold data for all epochs for the current channel
        try:
            file_paths = find_edf_files(channel_dir)
            if not file_paths:
                print(f"No .edf files found in {channel_dir}. Skipping this directory.")
                continue
            for file_path in file_paths:
                try:
                    epochs = mne.read_epochs(file_path, preload=True)
                    print(f"{file_path} contains {len(epochs)} epochs")
                    if info is None:
                        info = epochs.info  # We only need to set this once
                    channel_epochs_data.append(epochs.get_data())  # Append data for these epochs
                except Exception as e:
                    print(f"Failed to read epochs from {file_path}. Error: {e}")
                    continue
        except Exception as e:
            print(f"Error processing directory {channel_dir}: {e}")
            continue
        if channel_epochs_data:
            # Concatenate all epoch data for the current channel
            channel_basename = os.path.basename(channel_dir)
            total_epochs = sum(len(e) for e in channel_epochs_data)
            print(f"Channel {channel_basename} combined total epochs: {total_epochs}")
            channel_combined_data = np.concatenate(channel_epochs_data, axis=0)
            all_epochs_data.append(channel_combined_data)

    if all_epochs_data:
        # Combine data from all channels, adjusting axis as necessary
        data_combined = np.concatenate(all_epochs_data, axis=1)  # Adjust axis based on your data structure
        print(f"Total combined epochs across all channels: {data_combined.shape[0]}")
        print(f"Diagnostic: Combined data shape: {data_combined.shape}")
        
    else:
        print("No epochs were loaded. Please check file paths and formats.")
        continue

    # Now, create the combined epochs structure
    if info is not None and data_combined.size > 0:
        # mapped_ch_names = [electrode_name_mapping.get(ch_name, ch_name) for ch_name in ch_names]
        # Update info to match the combined data
        info = mne.create_info(ch_names=ch_names, sfreq=info['sfreq'], ch_types=['eeg'] * len(ch_names))
        # Create a dummy events array and event_id dict, as they are required for creating EpochsArray
        events = np.array([[i, 0, 1] for i in range(data_combined.shape[0])])
        event_id = {'dummy_event': 1}
        epochs_combined = mne.EpochsArray(data_combined, info, events=events, event_id=event_id)
        print(f"Starting connectivity analysis on combined data...")
        processed_epochs_count = 0
        save_path = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\coherence_visualization_depression\delta_band_coherence_depression"
        save_csv_path = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\csv_data_coherence_depression\delta_band_csv_depression"
        for epoch_idx in range(len(epochs_combined)):
            # Here you can proceed with your connectivity analysis on `epochs_combined`
            con = spectral_connectivity_epochs(
                epochs_combined[epoch_idx:epoch_idx+1], method='coh', mode='multitaper', sfreq=epochs_combined.info['sfreq'],
                fmin=0.5, fmax=4, faverage=True, mt_adaptive=True, n_jobs=1
            )
            con_matrix = con.get_data(output='dense')[..., 0]
            df_con = pd.DataFrame(con_matrix, index=ch_names, columns=ch_names)
            processed_epochs_count += 1
            
            csv_file_name = f"coherence_matrix_Depression_deltaBand_epoch_{epoch_idx + 1}.csv"
            csv_full_path = os.path.join(save_csv_path, csv_file_name)
            df_con.to_csv(csv_full_path)

            plt.figure(figsize=(10, 8))
            sns.heatmap(df_con, annot=True, cmap='viridis', fmt=".2f")
            plt.title('Coherence Matrix for Delta Band Depression')
            plt.ylabel('Channels')
            plt.xlabel('Channels')
            file_name = f"coherence_matrix_Depression_deltaBand_epoch_{epoch_idx + 1}.png"
            full_path = os.path.join(save_path, file_name)
            plt.savefig(full_path)
            plt.close()
            print(f"Processed and visualized {processed_epochs_count} epochs so far.")
            print(f"Total epochs processed for visualization: {processed_epochs_count}")
        else:
            print("No combined data available for creating epochs.")


## Gamma Band

In [ ]:
# electrode_name_mapping = {
#     'EEG039': 'FT7',
#     'EEG045': 'T7',
#     'EEG050': 'TP7',
#     'EEG101': 'TP8',
#     'EEG108': 'T8',
#     'EEG115': 'FT8',
#     'EEG024': 'F3',
#     'EEG124': 'F4'
# }

def find_fif_files(directory):
    return [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.fif')]

channel_groups = [
    [
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\filter_channel_FT7_depression\gamma_band_FT7_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\filter_channel_T7_depression\gamma_band_T7_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\filter_channel_TP7_depression\gamma_band_TP7_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\filter_channel_TP8_depression\gamma_band_TP8_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\filter_channel_T8_depression\gamma_band_T8_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\filter_channel_FT8_depression\gamma_band_FT8_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\filter_channel_F3_depression\gamma_band_F3_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\filter_channel_F4_depression\gamma_band_F4_depression"
    ]
]

for channel_group in channel_groups:
    all_epochs_data = []  # This will hold the data for all epochs across all channels
    ch_names = ['FT7', 'T7', 'TP7', 'TP8', 'T8', 'FT8', 'F3', 'F4']
    info = None  # This will be used to store the info structure for creating combined epochs later
    for channel_dir in channel_group:
        channel_epochs_data = []  # This will hold data for all epochs for the current channel
        try:
            file_paths = find_fif_files(channel_dir)
            if not file_paths:
                print(f"No .fif files found in {channel_dir}. Skipping this directory.")
                continue
            for file_path in file_paths:
                try:
                    epochs = mne.read_epochs(file_path, preload=True)
                    print(f"{file_path} contains {len(epochs)} epochs")
                    if info is None:
                        info = epochs.info  # We only need to set this once
                    channel_epochs_data.append(epochs.get_data())  # Append data for these epochs
                except Exception as e:
                    print(f"Failed to read epochs from {file_path}. Error: {e}")
                    continue
        except Exception as e:
            print(f"Error processing directory {channel_dir}: {e}")
            continue
        if channel_epochs_data:
            # Concatenate all epoch data for the current channel
            channel_basename = os.path.basename(channel_dir)
            total_epochs = sum(len(e) for e in channel_epochs_data)
            print(f"Channel {channel_basename} combined total epochs: {total_epochs}")
            channel_combined_data = np.concatenate(channel_epochs_data, axis=0)
            all_epochs_data.append(channel_combined_data)

    if all_epochs_data:
        # Combine data from all channels, adjusting axis as necessary
        data_combined = np.concatenate(all_epochs_data, axis=1)  # Adjust axis based on your data structure
        print(f"Total combined epochs across all channels: {data_combined.shape[0]}")
        print(f"Diagnostic: Combined data shape: {data_combined.shape}")
        
    else:
        print("No epochs were loaded. Please check file paths and formats.")
        continue

    # Now, create the combined epochs structure
    if info is not None and data_combined.size > 0:
        # mapped_ch_names = [electrode_name_mapping.get(ch_name, ch_name) for ch_name in ch_names]
        # Update info to match the combined data
        info = mne.create_info(ch_names=ch_names, sfreq=info['sfreq'], ch_types=['eeg'] * len(ch_names))
        # Create a dummy events array and event_id dict, as they are required for creating EpochsArray
        events = np.array([[i, 0, 1] for i in range(data_combined.shape[0])])
        event_id = {'dummy_event': 1}
        epochs_combined = mne.EpochsArray(data_combined, info, events=events, event_id=event_id)
        print(f"Starting connectivity analysis on combined data...")
        processed_epochs_count = 0
        save_path = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\coherence_visualization_depression\gamma_band_coherence_depression"
        save_csv_path = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\csv_data_depression\gamma_band_csv_depression"
        for epoch_idx in range(len(epochs_combined)):
            # Here you can proceed with your connectivity analysis on `epochs_combined`
            con = spectral_connectivity_epochs(
                epochs_combined[epoch_idx:epoch_idx+1], method='coh', mode='multitaper', sfreq=epochs_combined.info['sfreq'],
                fmin=30, fmax=40, faverage=True, mt_adaptive=True, n_jobs=1
            )
            con_matrix = con.get_data(output='dense')[..., 0]
            df_con = pd.DataFrame(con_matrix, index=ch_names, columns=ch_names)
            processed_epochs_count += 1
            
            csv_file_name = f"coherence_matrix_Depression_gammaBand_epoch_{epoch_idx + 1}.csv"
            csv_full_path = os.path.join(save_csv_path, csv_file_name)
            df_con.to_csv(csv_full_path)

            plt.figure(figsize=(10, 8))
            sns.heatmap(df_con, annot=True, cmap='viridis', fmt=".2f")
            plt.title('Coherence Matrix for Gamma Band Depression')
            plt.ylabel('Channels')
            plt.xlabel('Channels')
            file_name = f"coherence_matrix_Depression_gammaBand_epoch_{epoch_idx + 1}.png"
            full_path = os.path.join(save_path, file_name)
            plt.savefig(full_path)
            plt.close()
            print(f"Processed and visualized {processed_epochs_count} epochs so far.")
            print(f"Total epochs processed for visualization: {processed_epochs_count}")
        else:
            print("No combined data available for creating epochs.")


## Gamma banda (dataset real)

In [ ]:
# electrode_name_mapping = {
#     'EEG039': 'FT7',
#     'EEG045': 'T7',
#     'EEG050': 'TP7',
#     'EEG101': 'TP8',
#     'EEG108': 'T8',
#     'EEG115': 'FT8',
#     'EEG024': 'F3',
#     'EEG124': 'F4'
# }

def find_edf_files(directory):
    return [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.edf')]

channel_groups = [
    [
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\filter_channel_C3-R_depression\delta_band_C3-R_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\filter_channel_C4-R_depression\delta_band_C4-R_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\filter_channel_FP1-R_depression\delta_band_FP1-R_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\filter_channel_FP2-R_depression\delta_band_FP2-R_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\filter_channel_O1-R_depression\delta_band_O1-R_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\filter_channel_O2-R_depression\delta_band_O2-R_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\filter_channel_T3-R_depression\delta_band_T3-R_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\filter_channel_T4-R_depression\delta_band_T4-R_depression"
    ]
]

for channel_group in channel_groups:
    all_epochs_data = []  # This will hold the data for all epochs across all channels
    ch_names = ['EEG FP1-R', 'EEG FP2-R', 'EEG C3-R', 'EEG C4-R', 'EEG O1-R', 'EEG O2-R', 'EEG T3-R', 'EEG T4-R']
    info = None  # This will be used to store the info structure for creating combined epochs later
    for channel_dir in channel_group:
        channel_epochs_data = []  # This will hold data for all epochs for the current channel
        try:
            file_paths = find_edf_files(channel_dir)
            if not file_paths:
                print(f"No .edf files found in {channel_dir}. Skipping this directory.")
                continue
            for file_path in file_paths:
                try:
                    epochs = mne.read_epochs(file_path, preload=True)
                    print(f"{file_path} contains {len(epochs)} epochs")
                    if info is None:
                        info = epochs.info  # We only need to set this once
                    channel_epochs_data.append(epochs.get_data())  # Append data for these epochs
                except Exception as e:
                    print(f"Failed to read epochs from {file_path}. Error: {e}")
                    continue
        except Exception as e:
            print(f"Error processing directory {channel_dir}: {e}")
            continue
        if channel_epochs_data:
            # Concatenate all epoch data for the current channel
            channel_basename = os.path.basename(channel_dir)
            total_epochs = sum(len(e) for e in channel_epochs_data)
            print(f"Channel {channel_basename} combined total epochs: {total_epochs}")
            channel_combined_data = np.concatenate(channel_epochs_data, axis=0)
            all_epochs_data.append(channel_combined_data)

    if all_epochs_data:
        # Combine data from all channels, adjusting axis as necessary
        data_combined = np.concatenate(all_epochs_data, axis=1)  # Adjust axis based on your data structure
        print(f"Total combined epochs across all channels: {data_combined.shape[0]}")
        print(f"Diagnostic: Combined data shape: {data_combined.shape}")
        
    else:
        print("No epochs were loaded. Please check file paths and formats.")
        continue

    # Now, create the combined epochs structure
    if info is not None and data_combined.size > 0:
        # mapped_ch_names = [electrode_name_mapping.get(ch_name, ch_name) for ch_name in ch_names]
        # Update info to match the combined data
        info = mne.create_info(ch_names=ch_names, sfreq=info['sfreq'], ch_types=['eeg'] * len(ch_names))
        # Create a dummy events array and event_id dict, as they are required for creating EpochsArray
        events = np.array([[i, 0, 1] for i in range(data_combined.shape[0])])
        event_id = {'dummy_event': 1}
        epochs_combined = mne.EpochsArray(data_combined, info, events=events, event_id=event_id)
        print(f"Starting connectivity analysis on combined data...")
        processed_epochs_count = 0
        save_path = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\coherence_visualization_depression\gamma_band_coherence_depression"
        save_csv_path = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\csv_data_coherence_depression\gamma_band_csv_depression"
        for epoch_idx in range(len(epochs_combined)):
            # Here you can proceed with your connectivity analysis on `epochs_combined`
            con = spectral_connectivity_epochs(
                epochs_combined[epoch_idx:epoch_idx+1], method='coh', mode='multitaper', sfreq=epochs_combined.info['sfreq'],
                fmin=30, fmax=40, faverage=True, mt_adaptive=True, n_jobs=1
            )
            con_matrix = con.get_data(output='dense')[..., 0]
            df_con = pd.DataFrame(con_matrix, index=ch_names, columns=ch_names)
            processed_epochs_count += 1
            
            csv_file_name = f"coherence_matrix_Depression_gammaBand_epoch_{epoch_idx + 1}.csv"
            csv_full_path = os.path.join(save_csv_path, csv_file_name)
            df_con.to_csv(csv_full_path)

            plt.figure(figsize=(10, 8))
            sns.heatmap(df_con, annot=True, cmap='viridis', fmt=".2f")
            plt.title('Coherence Matrix for Gamma Band Depression')
            plt.ylabel('Channels')
            plt.xlabel('Channels')
            file_name = f"coherence_matrix_Depression_gammaBand_epoch_{epoch_idx + 1}.png"
            full_path = os.path.join(save_path, file_name)
            plt.savefig(full_path)
            plt.close()
            print(f"Processed and visualized {processed_epochs_count} epochs so far.")
            print(f"Total epochs processed for visualization: {processed_epochs_count}")
        else:
            print("No combined data available for creating epochs.")


## Theta Band

In [ ]:
# electrode_name_mapping = {
#     'EEG039': 'FT7',
#     'EEG045': 'T7',
#     'EEG050': 'TP7',
#     'EEG101': 'TP8',
#     'EEG108': 'T8',
#     'EEG115': 'FT8',
#     'EEG024': 'F3',
#     'EEG124': 'F4'
# }

def find_fif_files(directory):
    return [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.fif')]

channel_groups = [
    [
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\filter_channel_FT7_depression\theta_band_FT7_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\filter_channel_T7_depression\theta_band_T7_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\filter_channel_TP7_depression\theta_band_TP7_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\filter_channel_TP8_depression\theta_band_TP8_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\filter_channel_T8_depression\theta_band_T8_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\filter_channel_FT8_depression\theta_band_FT8_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\filter_channel_F3_depression\theta_band_F3_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\filter_channel_F4_depression\theta_band_F4_depression"
    ]
]

for channel_group in channel_groups:
    all_epochs_data = []  # This will hold the data for all epochs across all channels
    ch_names = ['FT7', 'T7', 'TP7', 'TP8', 'T8', 'FT8', 'F3', 'F4']
    info = None  # This will be used to store the info structure for creating combined epochs later
    for channel_dir in channel_group:
        channel_epochs_data = []  # This will hold data for all epochs for the current channel
        try:
            file_paths = find_fif_files(channel_dir)
            if not file_paths:
                print(f"No .fif files found in {channel_dir}. Skipping this directory.")
                continue
            for file_path in file_paths:
                try:
                    epochs = mne.read_epochs(file_path, preload=True)
                    print(f"{file_path} contains {len(epochs)} epochs")
                    if info is None:
                        info = epochs.info  # We only need to set this once
                    channel_epochs_data.append(epochs.get_data())  # Append data for these epochs
                except Exception as e:
                    print(f"Failed to read epochs from {file_path}. Error: {e}")
                    continue
        except Exception as e:
            print(f"Error processing directory {channel_dir}: {e}")
            continue
        if channel_epochs_data:
            # Concatenate all epoch data for the current channel
            channel_basename = os.path.basename(channel_dir)
            total_epochs = sum(len(e) for e in channel_epochs_data)
            print(f"Channel {channel_basename} combined total epochs: {total_epochs}")
            channel_combined_data = np.concatenate(channel_epochs_data, axis=0)
            all_epochs_data.append(channel_combined_data)

    if all_epochs_data:
        # Combine data from all channels, adjusting axis as necessary
        data_combined = np.concatenate(all_epochs_data, axis=1)  # Adjust axis based on your data structure
        print(f"Total combined epochs across all channels: {data_combined.shape[0]}")
        print(f"Diagnostic: Combined data shape: {data_combined.shape}")
        
    else:
        print("No epochs were loaded. Please check file paths and formats.")
        continue

    # Now, create the combined epochs structure
    if info is not None and data_combined.size > 0:
        # mapped_ch_names = [electrode_name_mapping.get(ch_name, ch_name) for ch_name in ch_names]
        # Update info to match the combined data
        info = mne.create_info(ch_names=ch_names, sfreq=info['sfreq'], ch_types=['eeg'] * len(ch_names))
        print(info)
        # Create a dummy events array and event_id dict, as they are required for creating EpochsArray
        events = np.array([[i, 0, 1] for i in range(data_combined.shape[0])])
        print(events)
        event_id = {'dummy_event': 1}
        epochs_combined = mne.EpochsArray(data_combined, info, events=events, event_id=event_id)
        data_epochs_combined_0 = epochs_combined.get_data()[0]
        print(data_epochs_combined_0.shape)
        print(f"Starting connectivity analysis on combined data...")
        processed_epochs_count = 0
        save_path = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\coherence_visualization_depression\theta_band_coherence_depression"
        save_csv_path = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\csv_data_depression\theta_band_csv_depression"
        for epoch_idx in range(len(epochs_combined)):
            # Here you can proceed with your connectivity analysis on `epochs_combined`
            con = spectral_connectivity_epochs(
                epochs_combined[epoch_idx:epoch_idx+1], method='coh', mode='multitaper', sfreq=epochs_combined.info['sfreq'],
                fmin=4, fmax=8, faverage=True, mt_adaptive=True, n_jobs=1
            )
            con_data = con.get_data()
            print(con_data)
            con_matrix = con.get_data(output='dense')[..., 0]
            df_con = pd.DataFrame(con_matrix, index=ch_names, columns=ch_names)
            print(df_con)
            processed_epochs_count += 1
            
            csv_file_name = f"coherence_matrix_Depression_thetaBand_epoch_{epoch_idx + 1}.csv"
            csv_full_path = os.path.join(save_csv_path, csv_file_name)
            df_con.to_csv(csv_full_path)
            
            plt.figure(figsize=(10, 8))
            sns.heatmap(df_con, annot=True, cmap='viridis', fmt=".2f")
            plt.title('Coherence Matrix for Theta Band Depression')
            plt.ylabel('Channels')
            plt.xlabel('Channels')
            file_name = f"coherence_matrix_Depression_thetaBand_epoch_{epoch_idx + 1}.png"
            full_path = os.path.join(save_path, file_name)
            plt.savefig(full_path)
            plt.close()
            print(f"Processed and visualized {processed_epochs_count} epochs so far.")
            print(f"Total epochs processed for visualization: {processed_epochs_count}")
        else:
            print("No combined data available for creating epochs.")


## Theta band (dataset real)

In [ ]:
# electrode_name_mapping = {
#     'EEG039': 'FT7',
#     'EEG045': 'T7',
#     'EEG050': 'TP7',
#     'EEG101': 'TP8',
#     'EEG108': 'T8',
#     'EEG115': 'FT8',
#     'EEG024': 'F3',
#     'EEG124': 'F4'
# }

def find_edf_files(directory):
    return [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.edf')]

channel_groups = [
    [
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\filter_channel_C3-R_depression\delta_band_C3-R_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\filter_channel_C4-R_depression\delta_band_C4-R_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\filter_channel_FP1-R_depression\delta_band_FP1-R_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\filter_channel_FP2-R_depression\delta_band_FP2-R_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\filter_channel_O1-R_depression\delta_band_O1-R_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\filter_channel_O2-R_depression\delta_band_O2-R_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\filter_channel_T3-R_depression\delta_band_T3-R_depression",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\filter_channel_T4-R_depression\delta_band_T4-R_depression"
    ]
]

for channel_group in channel_groups:
    all_epochs_data = []  # This will hold the data for all epochs across all channels
    ch_names = ['EEG FP1-R', 'EEG FP2-R', 'EEG C3-R', 'EEG C4-R', 'EEG O1-R', 'EEG O2-R', 'EEG T3-R', 'EEG T4-R']
    info = None  # This will be used to store the info structure for creating combined epochs later
    for channel_dir in channel_group:
        channel_epochs_data = []  # This will hold data for all epochs for the current channel
        try:
            file_paths = find_edf_files(channel_dir)
            if not file_paths:
                print(f"No .edf files found in {channel_dir}. Skipping this directory.")
                continue
            for file_path in file_paths:
                try:
                    epochs = mne.read_epochs(file_path, preload=True)
                    print(f"{file_path} contains {len(epochs)} epochs")
                    if info is None:
                        info = epochs.info  # We only need to set this once
                    channel_epochs_data.append(epochs.get_data())  # Append data for these epochs
                except Exception as e:
                    print(f"Failed to read epochs from {file_path}. Error: {e}")
                    continue
        except Exception as e:
            print(f"Error processing directory {channel_dir}: {e}")
            continue
        if channel_epochs_data:
            # Concatenate all epoch data for the current channel
            channel_basename = os.path.basename(channel_dir)
            total_epochs = sum(len(e) for e in channel_epochs_data)
            print(f"Channel {channel_basename} combined total epochs: {total_epochs}")
            channel_combined_data = np.concatenate(channel_epochs_data, axis=0)
            all_epochs_data.append(channel_combined_data)

    if all_epochs_data:
        # Combine data from all channels, adjusting axis as necessary
        data_combined = np.concatenate(all_epochs_data, axis=1)  # Adjust axis based on your data structure
        print(f"Total combined epochs across all channels: {data_combined.shape[0]}")
        print(f"Diagnostic: Combined data shape: {data_combined.shape}")
        
    else:
        print("No epochs were loaded. Please check file paths and formats.")
        continue

    # Now, create the combined epochs structure
    if info is not None and data_combined.size > 0:
        # mapped_ch_names = [electrode_name_mapping.get(ch_name, ch_name) for ch_name in ch_names]
        # Update info to match the combined data
        info = mne.create_info(ch_names=ch_names, sfreq=info['sfreq'], ch_types=['eeg'] * len(ch_names))
        # Create a dummy events array and event_id dict, as they are required for creating EpochsArray
        events = np.array([[i, 0, 1] for i in range(data_combined.shape[0])])
        event_id = {'dummy_event': 1}
        epochs_combined = mne.EpochsArray(data_combined, info, events=events, event_id=event_id)
        print(f"Starting connectivity analysis on combined data...")
        processed_epochs_count = 0
        save_path = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\coherence_visualization_depression\theta_band_coherence_depression"
        save_csv_path = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\csv_data_coherence_depression\theta_band_csv_depression"
        for epoch_idx in range(len(epochs_combined)):
            # Here you can proceed with your connectivity analysis on `epochs_combined`
            con = spectral_connectivity_epochs(
                epochs_combined[epoch_idx:epoch_idx+1], method='coh', mode='multitaper', sfreq=epochs_combined.info['sfreq'],
                fmin=4, fmax=8, faverage=True, mt_adaptive=True, n_jobs=1
            )
            con_matrix = con.get_data(output='dense')[..., 0]
            df_con = pd.DataFrame(con_matrix, index=ch_names, columns=ch_names)
            processed_epochs_count += 1
            
            csv_file_name = f"coherence_matrix_Depression_thetaBand_epoch_{epoch_idx + 1}.csv"
            csv_full_path = os.path.join(save_csv_path, csv_file_name)
            df_con.to_csv(csv_full_path)
            
            plt.figure(figsize=(10, 8))
            sns.heatmap(df_con, annot=True, cmap='viridis', fmt=".2f")
            plt.title('Coherence Matrix for Theta Band Depression')
            plt.ylabel('Channels')
            plt.xlabel('Channels')
            file_name = f"coherence_matrix_Depression_thetaBand_epoch_{epoch_idx + 1}.png"
            full_path = os.path.join(save_path, file_name)
            plt.savefig(full_path)
            plt.close()
            print(f"Processed and visualized {processed_epochs_count} epochs so far.")
            print(f"Total epochs processed for visualization: {processed_epochs_count}")
        else:
            print("No combined data available for creating epochs.")


# Correlation Matrix 
## Dataset depression

In [ ]:
# Metode Korelasi Pearson dataset healthy
# electrode_name_mapping = {
#     'EEG039': 'FT7',
#     'EEG115': 'FT8',
#     'EEG045': 'T7',
#     'EEG108': 'T8',
#     'EEG050': 'TP7',
#     'EEG101': 'TP8',
#     'EEG024': 'F3',
#     'EEG124': 'F4'
# }

directory_path = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\6s_epochs_depression"
selected_electrodes_names = ['FT7', 'T7', 'TP7', 'TP8', 'T8', 'FT8', 'F3', 'F4']
file_paths = [os.path.join(directory_path, file) for file in os.listdir(directory_path) if file.endswith(".fif")]
save_directory = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\correlation_visualization_depression"
save_csv_path = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Depression\csv_data_correlation_depression"
os.makedirs(save_directory, exist_ok=True)
os.makedirs(save_csv_path, exist_ok=True) 
total_epochs = 0
for file_path in file_paths:
    epochs = mne.read_epochs(file_path, preload=True)
    num_epochs = len(epochs)
    total_epochs += num_epochs
    file_basename = os.path.basename(file_path).split('.')[0]
    print(f"{os.path.basename(file_path)}: {num_epochs} epochs")

    for epoch_idx, epoch_data in enumerate(epochs.get_data()):
        selected_indices = [epochs.ch_names.index(name) for name in selected_electrodes_names if name in epochs.ch_names]
        selected_data = epoch_data[selected_indices]
        column_labels = [epochs.ch_names[idx] for idx in selected_indices]
        df = pd.DataFrame(selected_data.T, columns=column_labels)
        correlation_matrix = df.corr()
        
        csv_filename = f"{file_basename}_correlation_matrix_epoch_{epoch_idx + 1}.csv"
        csv_file_path = os.path.join(save_csv_path, csv_filename)
        correlation_matrix.to_csv(csv_file_path)

        plt.figure(figsize=(10, 8))
        sns.heatmap(correlation_matrix, annot=True, cmap="viridis", fmt=".2f", 
                    xticklabels=column_labels, 
                    yticklabels=column_labels)
        file_name = f"{file_basename}_correlation_matrix_epoch_{epoch_idx + 1}.png"
        plt.title(f"Pearson Correlation Matrix")
        save_path = os.path.join(save_directory, file_name) 
        plt.savefig(save_path)
        plt.close()
print(f"Total epochs loaded: {total_epochs}")

## Dataset depression (dataset real)

In [ ]:
# Metode Korelasi Pearson dataset healthy
# electrode_name_mapping = {
#     'EEG039': 'FT7',
#     'EEG115': 'FT8',
#     'EEG045': 'T7',
#     'EEG108': 'T8',
#     'EEG050': 'TP7',
#     'EEG101': 'TP8',
#     'EEG024': 'F3',
#     'EEG124': 'F4'
# }

directory_path = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\6s_epochs_depression"
selected_electrodes_names = ['EEG FP1-R', 'EEG FP2-R', 'EEG C3-R', 'EEG C4-R', 'EEG O1-R', 'EEG O2-R', 'EEG T3-R', 'EEG T4-R']
file_paths = [os.path.join(directory_path, file) for file in os.listdir(directory_path) if file.endswith(".edf")]
save_directory = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\correlation_visualization_depression"
save_csv_path = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Depression\csv_data_correlation_depression"
os.makedirs(save_directory, exist_ok=True)
os.makedirs(save_csv_path, exist_ok=True) 
total_epochs = 0
for file_path in file_paths:
    epochs = mne.read_epochs(file_path, preload=True)
    num_epochs = len(epochs)
    total_epochs += num_epochs
    file_basename = os.path.basename(file_path).split('.')[0]
    print(f"{os.path.basename(file_path)}: {num_epochs} epochs")

    for epoch_idx, epoch_data in enumerate(epochs.get_data()):
        selected_indices = [epochs.ch_names.index(name) for name in selected_electrodes_names if name in epochs.ch_names]
        selected_data = epoch_data[selected_indices]
        column_labels = [epochs.ch_names[idx] for idx in selected_indices]
        df = pd.DataFrame(selected_data.T, columns=column_labels)
        correlation_matrix = df.corr()
        
        csv_filename = f"{file_basename}_correlation_matrix_epoch_{epoch_idx + 1}.csv"
        csv_file_path = os.path.join(save_csv_path, csv_filename)
        correlation_matrix.to_csv(csv_file_path)

        plt.figure(figsize=(10, 8))
        sns.heatmap(correlation_matrix, annot=True, cmap="viridis", fmt=".2f", 
                    xticklabels=column_labels, 
                    yticklabels=column_labels)
        file_name = f"{file_basename}_correlation_matrix_epoch_{epoch_idx + 1}.png"
        plt.title(f"Pearson Correlation Matrix")
        save_path = os.path.join(save_directory, file_name) 
        plt.savefig(save_path)
        plt.close()
print(f"Total epochs loaded: {total_epochs}")

# Coherence Matrix for Healthy Dataset

## Alpha Band

In [ ]:
# electrode_name_mapping = {
#     'EEG039': 'FT7',
#     'EEG045': 'T7',
#     'EEG050': 'TP7',
#     'EEG101': 'TP8',
#     'EEG108': 'T8',
#     'EEG115': 'FT8',
#     'EEG024': 'F3',
#     'EEG124': 'F4'
# }

def find_fif_files(directory):
    return [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.fif')]

channel_groups = [
    [
       r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\filter_channel_FT7_healthy\alpha_band_FT7_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\filter_channel_T7_healthy\alpha_band_T7_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\filter_channel_TP7_healthy\alpha_band_TP7_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\filter_channel_TP8_healthy\alpha_band_TP8_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\filter_channel_T8_healthy\alpha_band_T8_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\filter_channel_FT8_healthy\alpha_band_FT8_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\filter_channel_F3_healthy\alpha_band_F3_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\filter_channel_F4_healthy\alpha_band_F4_healthy"
    ]
]

for channel_group in channel_groups:
    all_epochs_data = []  # This will hold the data for all epochs across all channels
    ch_names = ['FT7', 'T7', 'TP7', 'TP8', 'T8', 'FT8', 'F3', 'F4']
    info = None  # This will be used to store the info structure for creating combined epochs later
    for channel_dir in channel_group:
        channel_epochs_data = []  # This will hold data for all epochs for the current channel
        try:
            file_paths = find_fif_files(channel_dir)
            if not file_paths:
                print(f"No .fif files found in {channel_dir}. Skipping this directory.")
                continue
            for file_path in file_paths:
                try:
                    epochs = mne.read_epochs(file_path, preload=True)
                    print(f"{file_path} contains {len(epochs)} epochs")
                    if info is None:
                        info = epochs.info  # We only need to set this once
                    channel_epochs_data.append(epochs.get_data())  # Append data for these epochs
                except Exception as e:
                    print(f"Failed to read epochs from {file_path}. Error: {e}")
                    continue
        except Exception as e:
            print(f"Error processing directory {channel_dir}: {e}")
            continue
        if channel_epochs_data:
            # Concatenate all epoch data for the current channel
            channel_basename = os.path.basename(channel_dir)
            total_epochs = sum(len(e) for e in channel_epochs_data)
            print(f"Channel {channel_basename} combined total epochs: {total_epochs}")
            channel_combined_data = np.concatenate(channel_epochs_data, axis=0)
            all_epochs_data.append(channel_combined_data)

    if all_epochs_data:
        # Combine data from all channels, adjusting axis as necessary
        data_combined = np.concatenate(all_epochs_data, axis=1)  # Adjust axis based on your data structure
        print(f"Total combined epochs across all channels: {data_combined.shape[0]}")
        print(f"Diagnostic: Combined data shape: {data_combined.shape}")
        
    else:
        print("No epochs were loaded. Please check file paths and formats.")
        continue

    # Now, create the combined epochs structure
    if info is not None and data_combined.size > 0:
        # mapped_ch_names = [electrode_name_mapping.get(ch_name, ch_name) for ch_name in ch_names]
        # Update info to match the combined data
        info = mne.create_info(ch_names=ch_names, sfreq=info['sfreq'], ch_types=['eeg'] * len(ch_names))
        # Create a dummy events array and event_id dict, as they are required for creating EpochsArray
        events = np.array([[i, 0, 1] for i in range(data_combined.shape[0])])
        event_id = {'dummy_event': 1}
        epochs_combined = mne.EpochsArray(data_combined, info, events=events, event_id=event_id)
        print(f"Starting connectivity analysis on combined data...")
        processed_epochs_count = 0
        save_path = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\coherence_visualization_healthy\alpha_band_coherence_healthy"
        save_csv_path = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\csv_data_coherence_healthy\alpha_band_csv_healthy"
        for epoch_idx in range(len(epochs_combined)):
            # Here you can proceed with your connectivity analysis on `epochs_combined`
            con = spectral_connectivity_epochs(
                epochs_combined[epoch_idx:epoch_idx+1], method='coh', mode='multitaper', sfreq=epochs_combined.info['sfreq'],
                fmin=8, fmax=12, faverage=True, mt_adaptive=True, n_jobs=1
            )
            con_matrix = con.get_data(output='dense')[..., 0]
            df_con = pd.DataFrame(con_matrix, index=ch_names, columns=ch_names)
            processed_epochs_count += 1
            
            csv_file_name = f"coherence_matrix_Healthy_alphaBand_epoch_{epoch_idx + 1}.csv"
            csv_full_path = os.path.join(save_csv_path, csv_file_name)
            df_con.to_csv(csv_full_path)
            
            plt.figure(figsize=(10, 8))
            sns.heatmap(df_con, annot=True, cmap='viridis', fmt=".2f")
            plt.title('Coherence Matrix for Alpha Band Healthy')
            plt.ylabel('Channels')
            plt.xlabel('Channels')
            file_name = f"coherence_matrix_Healthy_alphaBand_epoch_{epoch_idx + 1}.png"
            full_path = os.path.join(save_path, file_name)
            plt.savefig(full_path)
            plt.close()
            print(f"Processed and visualized {processed_epochs_count} epochs so far.")
            print(f"Total epochs processed for visualization: {processed_epochs_count}")
        else:
            print("No combined data available for creating epochs.")


## Alpha band (dataset real)

In [ ]:
# electrode_name_mapping = {
#     'EEG039': 'FT7',
#     'EEG045': 'T7',
#     'EEG050': 'TP7',
#     'EEG101': 'TP8',
#     'EEG108': 'T8',
#     'EEG115': 'FT8',
#     'EEG024': 'F3',
#     'EEG124': 'F4'
# }

def find_edf_files(directory):
    return [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.edf')]

channel_groups = [
    [
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\filter_channel_C3-R_healthy\alpha_band_C3-R_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\filter_channel_C4-R_healthy\alpha_band_C4-R_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\filter_channel_FP1-R_healthy\alpha_band_FP1-R_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\filter_channel_FP2-R_healthy\alpha_band_FP2-R_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\filter_channel_O1-R_healthy\alpha_band_O1-R_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\filter_channel_O2-R_healthy\alpha_band_O2-R_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\filter_channel_T3-R_healthy\alpha_band_T3-R_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\filter_channel_T4-R_healthy\alpha_band_T4-R_healthy"
    ]
]

for channel_group in channel_groups:
    all_epochs_data = []  # This will hold the data for all epochs across all channels
    ch_names = ['EEG FP1-R', 'EEG FP2-R', 'EEG C3-R', 'EEG C4-R', 'EEG O1-R', 'EEG O2-R', 'EEG T3-R', 'EEG T4-R']
    info = None  # This will be used to store the info structure for creating combined epochs later
    for channel_dir in channel_group:
        channel_epochs_data = []  # This will hold data for all epochs for the current channel
        try:
            file_paths = find_edf_files(channel_dir)
            if not file_paths:
                print(f"No .edf files found in {channel_dir}. Skipping this directory.")
                continue
            for file_path in file_paths:
                try:
                    epochs = mne.read_epochs(file_path, preload=True)
                    print(f"{file_path} contains {len(epochs)} epochs")
                    if info is None:
                        info = epochs.info  # We only need to set this once
                    channel_epochs_data.append(epochs.get_data())  # Append data for these epochs
                except Exception as e:
                    print(f"Failed to read epochs from {file_path}. Error: {e}")
                    continue
        except Exception as e:
            print(f"Error processing directory {channel_dir}: {e}")
            continue
        if channel_epochs_data:
            # Concatenate all epoch data for the current channel
            channel_basename = os.path.basename(channel_dir)
            total_epochs = sum(len(e) for e in channel_epochs_data)
            print(f"Channel {channel_basename} combined total epochs: {total_epochs}")
            channel_combined_data = np.concatenate(channel_epochs_data, axis=0)
            all_epochs_data.append(channel_combined_data)

    if all_epochs_data:
        # Combine data from all channels, adjusting axis as necessary
        data_combined = np.concatenate(all_epochs_data, axis=1)  # Adjust axis based on your data structure
        print(f"Total combined epochs across all channels: {data_combined.shape[0]}")
        print(f"Diagnostic: Combined data shape: {data_combined.shape}")
        
    else:
        print("No epochs were loaded. Please check file paths and formats.")
        continue

    # Now, create the combined epochs structure
    if info is not None and data_combined.size > 0:
        # mapped_ch_names = [electrode_name_mapping.get(ch_name, ch_name) for ch_name in ch_names]
        # Update info to match the combined data
        info = mne.create_info(ch_names=ch_names, sfreq=info['sfreq'], ch_types=['eeg'] * len(ch_names))
        # Create a dummy events array and event_id dict, as they are required for creating EpochsArray
        events = np.array([[i, 0, 1] for i in range(data_combined.shape[0])])
        event_id = {'dummy_event': 1}
        epochs_combined = mne.EpochsArray(data_combined, info, events=events, event_id=event_id)
        print(f"Starting connectivity analysis on combined data...")
        processed_epochs_count = 0
        save_path = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\coherence_visualization_healthy\alpha_band_coherence_healthy"
        save_csv_path = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\csv_data_coherence_healthy\alpha_band_csv_healthy"
        for epoch_idx in range(len(epochs_combined)):
            # Here you can proceed with your connectivity analysis on `epochs_combined`
            con = spectral_connectivity_epochs(
                epochs_combined[epoch_idx:epoch_idx+1], method='coh', mode='multitaper', sfreq=epochs_combined.info['sfreq'],
                fmin=8, fmax=12, faverage=True, mt_adaptive=True, n_jobs=1
            )
            con_matrix = con.get_data(output='dense')[..., 0]
            df_con = pd.DataFrame(con_matrix, index=ch_names, columns=ch_names)
            processed_epochs_count += 1
            
            csv_file_name = f"coherence_matrix_Healthy_alphaBand_epoch_{epoch_idx + 1}.csv"
            csv_full_path = os.path.join(save_csv_path, csv_file_name)
            df_con.to_csv(csv_full_path)
            
            plt.figure(figsize=(10, 8))
            sns.heatmap(df_con, annot=True, cmap='viridis', fmt=".2f")
            plt.title('Coherence Matrix for Alpha Band Healthy')
            plt.ylabel('Channels')
            plt.xlabel('Channels')
            file_name = f"coherence_matrix_Healthy_alphaBand_epoch_{epoch_idx + 1}.png"
            full_path = os.path.join(save_path, file_name)
            plt.savefig(full_path)
            plt.close()
            print(f"Processed and visualized {processed_epochs_count} epochs so far.")
            print(f"Total epochs processed for visualization: {processed_epochs_count}")
        else:
            print("No combined data available for creating epochs.")


## Beta Band

In [ ]:
# electrode_name_mapping = {
#     'EEG039': 'FT7',
#     'EEG045': 'T7',
#     'EEG050': 'TP7',
#     'EEG101': 'TP8',
#     'EEG108': 'T8',
#     'EEG115': 'FT8',
#     'EEG024': 'F3',
#     'EEG124': 'F4'
# }

def find_fif_files(directory):
    return [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.fif')]

channel_groups = [
    [
       r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\filter_channel_FT7_healthy\beta_band_FT7_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\filter_channel_T7_healthy\beta_band_T7_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\filter_channel_TP7_healthy\beta_band_TP7_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\filter_channel_TP8_healthy\beta_band_TP8_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\filter_channel_T8_healthy\beta_band_T8_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\filter_channel_FT8_healthy\beta_band_FT8_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\filter_channel_F3_healthy\beta_band_F3_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\filter_channel_F4_healthy\beta_band_F4_healthy"
    ]
]

for channel_group in channel_groups:
    all_epochs_data = []  # This will hold the data for all epochs across all channels
    ch_names = ['FT7', 'T7', 'TP7', 'TP8', 'T8', 'FT8', 'F3', 'F4']
    info = None  # This will be used to store the info structure for creating combined epochs later
    for channel_dir in channel_group:
        channel_epochs_data = []  # This will hold data for all epochs for the current channel
        try:
            file_paths = find_fif_files(channel_dir)
            if not file_paths:
                print(f"No .fif files found in {channel_dir}. Skipping this directory.")
                continue
            for file_path in file_paths:
                try:
                    epochs = mne.read_epochs(file_path, preload=True)
                    print(f"{file_path} contains {len(epochs)} epochs")
                    if info is None:
                        info = epochs.info  # We only need to set this once
                    channel_epochs_data.append(epochs.get_data())  # Append data for these epochs
                except Exception as e:
                    print(f"Failed to read epochs from {file_path}. Error: {e}")
                    continue
        except Exception as e:
            print(f"Error processing directory {channel_dir}: {e}")
            continue
        if channel_epochs_data:
            # Concatenate all epoch data for the current channel
            channel_basename = os.path.basename(channel_dir)
            total_epochs = sum(len(e) for e in channel_epochs_data)
            print(f"Channel {channel_basename} combined total epochs: {total_epochs}")
            channel_combined_data = np.concatenate(channel_epochs_data, axis=0)
            all_epochs_data.append(channel_combined_data)

    if all_epochs_data:
        # Combine data from all channels, adjusting axis as necessary
        data_combined = np.concatenate(all_epochs_data, axis=1)  # Adjust axis based on your data structure
        print(f"Total combined epochs across all channels: {data_combined.shape[0]}")
        print(f"Diagnostic: Combined data shape: {data_combined.shape}")
        
    else:
        print("No epochs were loaded. Please check file paths and formats.")
        continue

    # Now, create the combined epochs structure
    if info is not None and data_combined.size > 0:
        # mapped_ch_names = [electrode_name_mapping.get(ch_name, ch_name) for ch_name in ch_names]
        # Update info to match the combined data
        info = mne.create_info(ch_names=ch_names, sfreq=info['sfreq'], ch_types=['eeg'] * len(ch_names))
        # Create a dummy events array and event_id dict, as they are required for creating EpochsArray
        events = np.array([[i, 0, 1] for i in range(data_combined.shape[0])])
        event_id = {'dummy_event': 1}
        epochs_combined = mne.EpochsArray(data_combined, info, events=events, event_id=event_id)
        print(f"Starting connectivity analysis on combined data...")
        processed_epochs_count = 0
        save_path = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\coherence_visualization_healthy\beta_band_coherence_healthy"
        save_csv_path = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\csv_data_coherence_healthy\beta_band_csv_healthy"
        for epoch_idx in range(len(epochs_combined)):
            # Here you can proceed with your connectivity analysis on `epochs_combined`
            con = spectral_connectivity_epochs(
                epochs_combined[epoch_idx:epoch_idx+1], method='coh', mode='multitaper', sfreq=epochs_combined.info['sfreq'],
                fmin=12, fmax=30, faverage=True, mt_adaptive=True, n_jobs=1
            )
            con_matrix = con.get_data(output='dense')[..., 0]
            df_con = pd.DataFrame(con_matrix, index=ch_names, columns=ch_names)
            processed_epochs_count += 1
            
            csv_file_name = f"coherence_matrix_Healthy_thetaBand_epoch_{epoch_idx + 1}.csv"
            csv_full_path = os.path.join(save_csv_path, csv_file_name)
            df_con.to_csv(csv_full_path)

            plt.figure(figsize=(10, 8))
            sns.heatmap(df_con, annot=True, cmap='viridis', fmt=".2f")
            plt.title('Coherence Matrix for Beta Band Healthy')
            plt.ylabel('Channels')
            plt.xlabel('Channels')
            file_name = f"coherence_matrix_Healthy_betaBand_epoch_{epoch_idx + 1}.png"
            full_path = os.path.join(save_path, file_name)
            plt.savefig(full_path)
            plt.close()
            print(f"Processed and visualized {processed_epochs_count} epochs so far.")
            print(f"Total epochs processed for visualization: {processed_epochs_count}")
        else:
            print("No combined data available for creating epochs.")


## Beta band (dataset real)

In [ ]:
# electrode_name_mapping = {
#     'EEG039': 'FT7',
#     'EEG045': 'T7',
#     'EEG050': 'TP7',
#     'EEG101': 'TP8',
#     'EEG108': 'T8',
#     'EEG115': 'FT8',
#     'EEG024': 'F3',
#     'EEG124': 'F4'
# }

def find_edf_files(directory):
    return [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.edf')]

channel_groups = [
    [
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\filter_channel_C3-R_healthy\beta_band_C3-R_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\filter_channel_C4-R_healthy\beta_band_C4-R_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\filter_channel_FP1-R_healthy\beta_band_FP1-R_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\filter_channel_FP2-R_healthy\beta_band_FP2-R_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\filter_channel_O1-R_healthy\beta_band_O1-R_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\filter_channel_O2-R_healthy\beta_band_O2-R_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\filter_channel_T3-R_healthy\beta_band_T3-R_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\filter_channel_T4-R_healthy\beta_band_T4-R_healthy"
    ]
]

for channel_group in channel_groups:
    all_epochs_data = []  # This will hold the data for all epochs across all channels
    ch_names = ['EEG FP1-R', 'EEG FP2-R', 'EEG C3-R', 'EEG C4-R', 'EEG O1-R', 'EEG O2-R', 'EEG T3-R', 'EEG T4-R']
    info = None  # This will be used to store the info structure for creating combined epochs later
    for channel_dir in channel_group:
        channel_epochs_data = []  # This will hold data for all epochs for the current channel
        try:
            file_paths = find_edf_files(channel_dir)
            if not file_paths:
                print(f"No .edf files found in {channel_dir}. Skipping this directory.")
                continue
            for file_path in file_paths:
                try:
                    epochs = mne.read_epochs(file_path, preload=True)
                    print(f"{file_path} contains {len(epochs)} epochs")
                    if info is None:
                        info = epochs.info  # We only need to set this once
                    channel_epochs_data.append(epochs.get_data())  # Append data for these epochs
                except Exception as e:
                    print(f"Failed to read epochs from {file_path}. Error: {e}")
                    continue
        except Exception as e:
            print(f"Error processing directory {channel_dir}: {e}")
            continue
        if channel_epochs_data:
            # Concatenate all epoch data for the current channel
            channel_basename = os.path.basename(channel_dir)
            total_epochs = sum(len(e) for e in channel_epochs_data)
            print(f"Channel {channel_basename} combined total epochs: {total_epochs}")
            channel_combined_data = np.concatenate(channel_epochs_data, axis=0)
            all_epochs_data.append(channel_combined_data)

    if all_epochs_data:
        # Combine data from all channels, adjusting axis as necessary
        data_combined = np.concatenate(all_epochs_data, axis=1)  # Adjust axis based on your data structure
        print(f"Total combined epochs across all channels: {data_combined.shape[0]}")
        print(f"Diagnostic: Combined data shape: {data_combined.shape}")
        
    else:
        print("No epochs were loaded. Please check file paths and formats.")
        continue

    # Now, create the combined epochs structure
    if info is not None and data_combined.size > 0:
        # mapped_ch_names = [electrode_name_mapping.get(ch_name, ch_name) for ch_name in ch_names]
        # Update info to match the combined data
        info = mne.create_info(ch_names=ch_names, sfreq=info['sfreq'], ch_types=['eeg'] * len(ch_names))
        # Create a dummy events array and event_id dict, as they are required for creating EpochsArray
        events = np.array([[i, 0, 1] for i in range(data_combined.shape[0])])
        event_id = {'dummy_event': 1}
        epochs_combined = mne.EpochsArray(data_combined, info, events=events, event_id=event_id)
        print(f"Starting connectivity analysis on combined data...")
        processed_epochs_count = 0
        save_path = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\coherence_visualization_healthy\beta_band_coherence_healthy"
        save_csv_path = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\csv_data_coherence_healthy\beta_band_csv_healthy"
        for epoch_idx in range(len(epochs_combined)):
            # Here you can proceed with your connectivity analysis on `epochs_combined`
            con = spectral_connectivity_epochs(
                epochs_combined[epoch_idx:epoch_idx+1], method='coh', mode='multitaper', sfreq=epochs_combined.info['sfreq'],
                fmin=12, fmax=30, faverage=True, mt_adaptive=True, n_jobs=1
            )
            con_matrix = con.get_data(output='dense')[..., 0]
            df_con = pd.DataFrame(con_matrix, index=ch_names, columns=ch_names)
            processed_epochs_count += 1
            
            csv_file_name = f"coherence_matrix_Healthy_thetaBand_epoch_{epoch_idx + 1}.csv"
            csv_full_path = os.path.join(save_csv_path, csv_file_name)
            df_con.to_csv(csv_full_path)

            plt.figure(figsize=(10, 8))
            sns.heatmap(df_con, annot=True, cmap='viridis', fmt=".2f")
            plt.title('Coherence Matrix for Beta Band Healthy')
            plt.ylabel('Channels')
            plt.xlabel('Channels')
            file_name = f"coherence_matrix_Healthy_betaBand_epoch_{epoch_idx + 1}.png"
            full_path = os.path.join(save_path, file_name)
            plt.savefig(full_path)
            plt.close()
            print(f"Processed and visualized {processed_epochs_count} epochs so far.")
            print(f"Total epochs processed for visualization: {processed_epochs_count}")
        else:
            print("No combined data available for creating epochs.")


## Delta Band

In [ ]:
# electrode_name_mapping = {
#     'EEG039': 'FT7',
#     'EEG045': 'T7',
#     'EEG050': 'TP7',
#     'EEG101': 'TP8',
#     'EEG108': 'T8',
#     'EEG115': 'FT8',
#     'EEG024': 'F3',
#     'EEG124': 'F4'
# }

def find_fif_files(directory):
    return [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.fif')]

channel_groups = [
    [
       r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\filter_channel_FT7_healthy\delta_band_FT7_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\filter_channel_T7_healthy\delta_band_T7_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\filter_channel_TP7_healthy\delta_band_TP7_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\filter_channel_TP8_healthy\delta_band_TP8_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\filter_channel_T8_healthy\delta_band_T8_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\filter_channel_FT8_healthy\delta_band_FT8_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\filter_channel_F3_healthy\delta_band_F3_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\filter_channel_F4_healthy\delta_band_F4_healthy"
    ]
]

for channel_group in channel_groups:
    all_epochs_data = []  # This will hold the data for all epochs across all channels
    ch_names = ['FT7', 'T7', 'TP7', 'TP8', 'T8', 'FT8', 'F3', 'F4']
    info = None  # This will be used to store the info structure for creating combined epochs later
    for channel_dir in channel_group:
        channel_epochs_data = []  # This will hold data for all epochs for the current channel
        try:
            file_paths = find_fif_files(channel_dir)
            if not file_paths:
                print(f"No .fif files found in {channel_dir}. Skipping this directory.")
                continue
            for file_path in file_paths:
                try:
                    epochs = mne.read_epochs(file_path, preload=True)
                    print(f"{file_path} contains {len(epochs)} epochs")
                    if info is None:
                        info = epochs.info  # We only need to set this once
                    channel_epochs_data.append(epochs.get_data())  # Append data for these epochs
                except Exception as e:
                    print(f"Failed to read epochs from {file_path}. Error: {e}")
                    continue
        except Exception as e:
            print(f"Error processing directory {channel_dir}: {e}")
            continue
        if channel_epochs_data:
            # Concatenate all epoch data for the current channel
            channel_basename = os.path.basename(channel_dir)
            total_epochs = sum(len(e) for e in channel_epochs_data)
            print(f"Channel {channel_basename} combined total epochs: {total_epochs}")
            channel_combined_data = np.concatenate(channel_epochs_data, axis=0)
            all_epochs_data.append(channel_combined_data)

    if all_epochs_data:
        # Combine data from all channels, adjusting axis as necessary
        data_combined = np.concatenate(all_epochs_data, axis=1)  # Adjust axis based on your data structure
        print(f"Total combined epochs across all channels: {data_combined.shape[0]}")
        print(f"Diagnostic: Combined data shape: {data_combined.shape}")
        
    else:
        print("No epochs were loaded. Please check file paths and formats.")
        continue

    # Now, create the combined epochs structure
    if info is not None and data_combined.size > 0:
        # mapped_ch_names = [electrode_name_mapping.get(ch_name, ch_name) for ch_name in ch_names]
        # Update info to match the combined data
        info = mne.create_info(ch_names=ch_names, sfreq=info['sfreq'], ch_types=['eeg'] * len(ch_names))
        # Create a dummy events array and event_id dict, as they are required for creating EpochsArray
        events = np.array([[i, 0, 1] for i in range(data_combined.shape[0])])
        event_id = {'dummy_event': 1}
        epochs_combined = mne.EpochsArray(data_combined, info, events=events, event_id=event_id)
        print(f"Starting connectivity analysis on combined data...")
        processed_epochs_count = 0
        save_path = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\coherence_visualization_healthy\delta_band_coherence_healthy"
        save_csv_path = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\csv_data_coherence_healthy\delta_band_csv_healthy"
        for epoch_idx in range(len(epochs_combined)):
            # Here you can proceed with your connectivity analysis on `epochs_combined`
            con = spectral_connectivity_epochs(
                epochs_combined[epoch_idx:epoch_idx+1], method='coh', mode='multitaper', sfreq=epochs_combined.info['sfreq'],
                fmin=0.1, fmax=3, faverage=True, mt_adaptive=True, n_jobs=1
            )
            con_matrix = con.get_data(output='dense')[..., 0]
            df_con = pd.DataFrame(con_matrix, index=ch_names, columns=ch_names)
            processed_epochs_count += 1

            csv_file_name = f"coherence_matrix_Healthy_deltaBand_epoch_{epoch_idx + 1}.csv"
            csv_full_path = os.path.join(save_csv_path, csv_file_name)
            df_con.to_csv(csv_full_path)

            plt.figure(figsize=(10, 8))
            sns.heatmap(df_con, annot=True, cmap='viridis', fmt=".2f")
            plt.title('Coherence Matrix for Delta Band Healthy')
            plt.ylabel('Channels')
            plt.xlabel('Channels')
            file_name = f"coherence_matrix_Healthy_deltaBand_epoch_{epoch_idx + 1}.png"
            full_path = os.path.join(save_path, file_name)
            plt.savefig(full_path)
            plt.close()
            print(f"Processed and visualized {processed_epochs_count} epochs so far.")
            print(f"Total epochs processed for visualization: {processed_epochs_count}")
        else:
            print("No combined data available for creating epochs.")

## Delta band (dataset real)

In [ ]:
# electrode_name_mapping = {
#     'EEG039': 'FT7',
#     'EEG045': 'T7',
#     'EEG050': 'TP7',
#     'EEG101': 'TP8',
#     'EEG108': 'T8',
#     'EEG115': 'FT8',
#     'EEG024': 'F3',
#     'EEG124': 'F4'
# }

def find_edf_files(directory):
    return [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.edf')]

channel_groups = [
    [
       r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\filter_channel_C3-R_healthy\delta_band_C3-R_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\filter_channel_C4-R_healthy\delta_band_C4-R_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\filter_channel_FP1-R_healthy\delta_band_FP1-R_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\filter_channel_FP2-R_healthy\delta_band_FP2-R_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\filter_channel_O1-R_healthy\delta_band_O1-R_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\filter_channel_O2-R_healthy\delta_band_O2-R_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\filter_channel_T3-R_healthy\delta_band_T3-R_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\filter_channel_T4-R_healthy\delta_band_T4-R_healthy"
    ]
]

for channel_group in channel_groups:
    all_epochs_data = []  # This will hold the data for all epochs across all channels
    ch_names = ['EEG FP1-R', 'EEG FP2-R', 'EEG C3-R', 'EEG C4-R', 'EEG O1-R', 'EEG O2-R', 'EEG T3-R', 'EEG T4-R']
    info = None  # This will be used to store the info structure for creating combined epochs later
    for channel_dir in channel_group:
        channel_epochs_data = []  # This will hold data for all epochs for the current channel
        try:
            file_paths = find_edf_files(channel_dir)
            if not file_paths:
                print(f"No .edf files found in {channel_dir}. Skipping this directory.")
                continue
            for file_path in file_paths:
                try:
                    epochs = mne.read_epochs(file_path, preload=True)
                    print(f"{file_path} contains {len(epochs)} epochs")
                    if info is None:
                        info = epochs.info  # We only need to set this once
                    channel_epochs_data.append(epochs.get_data())  # Append data for these epochs
                except Exception as e:
                    print(f"Failed to read epochs from {file_path}. Error: {e}")
                    continue
        except Exception as e:
            print(f"Error processing directory {channel_dir}: {e}")
            continue
        if channel_epochs_data:
            # Concatenate all epoch data for the current channel
            channel_basename = os.path.basename(channel_dir)
            total_epochs = sum(len(e) for e in channel_epochs_data)
            print(f"Channel {channel_basename} combined total epochs: {total_epochs}")
            channel_combined_data = np.concatenate(channel_epochs_data, axis=0)
            all_epochs_data.append(channel_combined_data)

    if all_epochs_data:
        # Combine data from all channels, adjusting axis as necessary
        data_combined = np.concatenate(all_epochs_data, axis=1)  # Adjust axis based on your data structure
        print(f"Total combined epochs across all channels: {data_combined.shape[0]}")
        print(f"Diagnostic: Combined data shape: {data_combined.shape}")
        
    else:
        print("No epochs were loaded. Please check file paths and formats.")
        continue

    # Now, create the combined epochs structure
    if info is not None and data_combined.size > 0:
        # mapped_ch_names = [electrode_name_mapping.get(ch_name, ch_name) for ch_name in ch_names]
        # Update info to match the combined data
        info = mne.create_info(ch_names=ch_names, sfreq=info['sfreq'], ch_types=['eeg'] * len(ch_names))
        # Create a dummy events array and event_id dict, as they are required for creating EpochsArray
        events = np.array([[i, 0, 1] for i in range(data_combined.shape[0])])
        event_id = {'dummy_event': 1}
        epochs_combined = mne.EpochsArray(data_combined, info, events=events, event_id=event_id)
        print(f"Starting connectivity analysis on combined data...")
        processed_epochs_count = 0
        save_path = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\coherence_visualization_healthy\delta_band_coherence_healthy"
        save_csv_path = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\csv_data_coherence_healthy\delta_band_csv_healthy"
        for epoch_idx in range(len(epochs_combined)):
            # Here you can proceed with your connectivity analysis on `epochs_combined`
            con = spectral_connectivity_epochs(
                epochs_combined[epoch_idx:epoch_idx+1], method='coh', mode='multitaper', sfreq=epochs_combined.info['sfreq'],
                fmin=0.1, fmax=3, faverage=True, mt_adaptive=True, n_jobs=1
            )
            con_matrix = con.get_data(output='dense')[..., 0]
            df_con = pd.DataFrame(con_matrix, index=ch_names, columns=ch_names)
            processed_epochs_count += 1

            csv_file_name = f"coherence_matrix_Healthy_deltaBand_epoch_{epoch_idx + 1}.csv"
            csv_full_path = os.path.join(save_csv_path, csv_file_name)
            df_con.to_csv(csv_full_path)

            plt.figure(figsize=(10, 8))
            sns.heatmap(df_con, annot=True, cmap='viridis', fmt=".2f")
            plt.title('Coherence Matrix for Delta Band Healthy')
            plt.ylabel('Channels')
            plt.xlabel('Channels')
            file_name = f"coherence_matrix_Healthy_deltaBand_epoch_{epoch_idx + 1}.png"
            full_path = os.path.join(save_path, file_name)
            plt.savefig(full_path)
            plt.close()
            
            print(f"Processed and visualized {processed_epochs_count} epochs so far.")
            print(f"Total epochs processed for visualization: {processed_epochs_count}")
        else:
            print("No combined data available for creating epochs.")

## Gamma Band

In [ ]:
# electrode_name_mapping = {
#     'EEG039': 'FT7',
#     'EEG045': 'T7',
#     'EEG050': 'TP7',
#     'EEG101': 'TP8',
#     'EEG108': 'T8',
#     'EEG115': 'FT8',
#     'EEG024': 'F3',
#     'EEG124': 'F4'
# }

def find_fif_files(directory):
    return [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.fif')]

channel_groups = [
    [
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\filter_channel_FT7_healthy\gamma_band_FT7_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\filter_channel_T7_healthy\gamma_band_T7_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\filter_channel_TP7_healthy\gamma_band_TP7_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\filter_channel_TP8_healthy\gamma_band_TP8_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\filter_channel_T8_healthy\gamma_band_T8_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\filter_channel_FT8_healthy\gamma_band_FT8_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\filter_channel_F3_healthy\gamma_band_F3_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\filter_channel_F4_healthy\gamma_band_F4_healthy"
    ]
]

for channel_group in channel_groups:
    all_epochs_data = []  # This will hold the data for all epochs across all channels
    ch_names = ['FT7', 'T7', 'TP7', 'TP8', 'T8', 'FT8', 'F3', 'F4']
    info = None  # This will be used to store the info structure for creating combined epochs later
    for channel_dir in channel_group:
        channel_epochs_data = []  # This will hold data for all epochs for the current channel
        try:
            file_paths = find_fif_files(channel_dir)
            if not file_paths:
                print(f"No .fif files found in {channel_dir}. Skipping this directory.")
                continue
            for file_path in file_paths:
                try:
                    epochs = mne.read_epochs(file_path, preload=True)
                    print(f"{file_path} contains {len(epochs)} epochs")
                    if info is None:
                        info = epochs.info  # We only need to set this once
                    channel_epochs_data.append(epochs.get_data())  # Append data for these epochs
                except Exception as e:
                    print(f"Failed to read epochs from {file_path}. Error: {e}")
                    continue
        except Exception as e:
            print(f"Error processing directory {channel_dir}: {e}")
            continue
        if channel_epochs_data:
            # Concatenate all epoch data for the current channel
            channel_basename = os.path.basename(channel_dir)
            total_epochs = sum(len(e) for e in channel_epochs_data)
            print(f"Channel {channel_basename} combined total epochs: {total_epochs}")
            channel_combined_data = np.concatenate(channel_epochs_data, axis=0)
            all_epochs_data.append(channel_combined_data)

    if all_epochs_data:
        # Combine data from all channels, adjusting axis as necessary
        data_combined = np.concatenate(all_epochs_data, axis=1)  # Adjust axis based on your data structure
        print(f"Total combined epochs across all channels: {data_combined.shape[0]}")
        print(f"Diagnostic: Combined data shape: {data_combined.shape}")
        
    else:
        print("No epochs were loaded. Please check file paths and formats.")
        continue

    # Now, create the combined epochs structure
    if info is not None and data_combined.size > 0:
        # mapped_ch_names = [electrode_name_mapping.get(ch_name, ch_name) for ch_name in ch_names]
        # Update info to match the combined data
        info = mne.create_info(ch_names=ch_names, sfreq=info['sfreq'], ch_types=['eeg'] * len(ch_names))
        # Create a dummy events array and event_id dict, as they are required for creating EpochsArray
        events = np.array([[i, 0, 1] for i in range(data_combined.shape[0])])
        event_id = {'dummy_event': 1}
        epochs_combined = mne.EpochsArray(data_combined, info, events=events, event_id=event_id)
        print(f"Starting connectivity analysis on combined data...")
        processed_epochs_count = 0
        save_path = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\coherence_visualization_healthy\gamma_band_coherence_healthy"
        save_csv_path = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\csv_data_coherence_healthy\gamma_band_csv_healthy"
        for epoch_idx in range(len(epochs_combined)):
            # Here you can proceed with your connectivity analysis on `epochs_combined`
            con = spectral_connectivity_epochs(
                epochs_combined[epoch_idx:epoch_idx+1], method='coh', mode='multitaper', sfreq=epochs_combined.info['sfreq'],
                fmin=30, fmax=45, faverage=True, mt_adaptive=True, n_jobs=1
            )
            con_matrix = con.get_data(output='dense')[..., 0]
            df_con = pd.DataFrame(con_matrix, index=ch_names, columns=ch_names)
            processed_epochs_count += 1

            csv_file_name = f"coherence_matrix_Healthy_gammaBand_epoch_{epoch_idx + 1}.csv"
            csv_full_path = os.path.join(save_csv_path, csv_file_name)
            df_con.to_csv(csv_full_path)

            plt.figure(figsize=(10, 8))
            sns.heatmap(df_con, annot=True, cmap='viridis', fmt=".2f")
            plt.title('Coherence Matrix for Gamma Band Healthy')
            plt.ylabel('Channels')
            plt.xlabel('Channels')
            file_name = f"coherence_matrix_Healthy_gammaBand_epoch_{epoch_idx + 1}.png"
            full_path = os.path.join(save_path, file_name)
            plt.savefig(full_path)
            plt.close()
            print(f"Processed and visualized {processed_epochs_count} epochs so far.")
            print(f"Total epochs processed for visualization: {processed_epochs_count}")
        else:
            print("No combined data available for creating epochs.")

## Gamma band (dataset real)

In [ ]:
# electrode_name_mapping = {
#     'EEG039': 'FT7',
#     'EEG045': 'T7',
#     'EEG050': 'TP7',
#     'EEG101': 'TP8',
#     'EEG108': 'T8',
#     'EEG115': 'FT8',
#     'EEG024': 'F3',
#     'EEG124': 'F4'
# }

def find_edf_files(directory):
    return [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.edf')]

channel_groups = [
    [
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\filter_channel_C3-R_healthy\gamma_band_C3-R_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\filter_channel_C4-R_healthy\gamma_band_C4-R_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\filter_channel_FP1-R_healthy\gamma_band_FP1-R_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\filter_channel_FP2-R_healthy\gamma_band_FP2-R_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\filter_channel_O1-R_healthy\gamma_band_O1-R_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\filter_channel_O2-R_healthy\gamma_band_O2-R_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\filter_channel_T3-R_healthy\gamma_band_T3-R_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\filter_channel_T4-R_healthy\gamma_band_T4-R_healthy"
    ]
]

for channel_group in channel_groups:
    all_epochs_data = []  # This will hold the data for all epochs across all channels
    ch_names = ['EEG FP1-R', 'EEG FP2-R', 'EEG C3-R', 'EEG C4-R', 'EEG O1-R', 'EEG O2-R', 'EEG T3-R', 'EEG T4-R']
    info = None  # This will be used to store the info structure for creating combined epochs later
    for channel_dir in channel_group:
        channel_epochs_data = []  # This will hold data for all epochs for the current channel
        try:
            file_paths = find_edf_files(channel_dir)
            if not file_paths:
                print(f"No .edf files found in {channel_dir}. Skipping this directory.")
                continue
            for file_path in file_paths:
                try:
                    epochs = mne.read_epochs(file_path, preload=True)
                    print(f"{file_path} contains {len(epochs)} epochs")
                    if info is None:
                        info = epochs.info  # We only need to set this once
                    channel_epochs_data.append(epochs.get_data())  # Append data for these epochs
                except Exception as e:
                    print(f"Failed to read epochs from {file_path}. Error: {e}")
                    continue
        except Exception as e:
            print(f"Error processing directory {channel_dir}: {e}")
            continue
        if channel_epochs_data:
            # Concatenate all epoch data for the current channel
            channel_basename = os.path.basename(channel_dir)
            total_epochs = sum(len(e) for e in channel_epochs_data)
            print(f"Channel {channel_basename} combined total epochs: {total_epochs}")
            channel_combined_data = np.concatenate(channel_epochs_data, axis=0)
            all_epochs_data.append(channel_combined_data)

    if all_epochs_data:
        # Combine data from all channels, adjusting axis as necessary
        data_combined = np.concatenate(all_epochs_data, axis=1)  # Adjust axis based on your data structure
        print(f"Total combined epochs across all channels: {data_combined.shape[0]}")
        print(f"Diagnostic: Combined data shape: {data_combined.shape}")
        
    else:
        print("No epochs were loaded. Please check file paths and formats.")
        continue

    # Now, create the combined epochs structure
    if info is not None and data_combined.size > 0:
        # mapped_ch_names = [electrode_name_mapping.get(ch_name, ch_name) for ch_name in ch_names]
        # Update info to match the combined data
        info = mne.create_info(ch_names=ch_names, sfreq=info['sfreq'], ch_types=['eeg'] * len(ch_names))
        # Create a dummy events array and event_id dict, as they are required for creating EpochsArray
        events = np.array([[i, 0, 1] for i in range(data_combined.shape[0])])
        event_id = {'dummy_event': 1}
        epochs_combined = mne.EpochsArray(data_combined, info, events=events, event_id=event_id)
        print(f"Starting connectivity analysis on combined data...")
        processed_epochs_count = 0
        save_path = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\coherence_visualization_healthy\gamma_band_coherence_healthy"
        save_csv_path = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\csv_data_coherence_healthy\gamma_band_csv_healthy"
        for epoch_idx in range(len(epochs_combined)):
            # Here you can proceed with your connectivity analysis on `epochs_combined`
            con = spectral_connectivity_epochs(
                epochs_combined[epoch_idx:epoch_idx+1], method='coh', mode='multitaper', sfreq=epochs_combined.info['sfreq'],
                fmin=30, fmax=45, faverage=True, mt_adaptive=True, n_jobs=1
            )
            con_matrix = con.get_data(output='dense')[..., 0]
            df_con = pd.DataFrame(con_matrix, index=ch_names, columns=ch_names)
            processed_epochs_count += 1

            csv_file_name = f"coherence_matrix_Healthy_gammaBand_epoch_{epoch_idx + 1}.csv"
            csv_full_path = os.path.join(save_csv_path, csv_file_name)
            df_con.to_csv(csv_full_path)

            plt.figure(figsize=(10, 8))
            sns.heatmap(df_con, annot=True, cmap='viridis', fmt=".2f")
            plt.title('Coherence Matrix for Gamma Band Healthy')
            plt.ylabel('Channels')
            plt.xlabel('Channels')
            file_name = f"coherence_matrix_Healthy_gammaBand_epoch_{epoch_idx + 1}.png"
            full_path = os.path.join(save_path, file_name)
            plt.savefig(full_path)
            plt.close()
            print(f"Processed and visualized {processed_epochs_count} epochs so far.")
            print(f"Total epochs processed for visualization: {processed_epochs_count}")
        else:
            print("No combined data available for creating epochs.")

## Theta band

In [ ]:
# electrode_name_mapping = {
#     'EEG039': 'FT7',
#     'EEG045': 'T7',
#     'EEG050': 'TP7',
#     'EEG101': 'TP8',
#     'EEG108': 'T8',
#     'EEG115': 'FT8',
#     'EEG024': 'F3',
#     'EEG124': 'F4'
# }

def find_fif_files(directory):
    return [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.fif')]

channel_groups = [
    [
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\filter_channel_FT7_healthy\theta_band_FT7_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\filter_channel_T7_healthy\theta_band_T7_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\filter_channel_TP7_healthy\theta_band_TP7_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\filter_channel_TP8_healthy\theta_band_TP8_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\filter_channel_T8_healthy\theta_band_T8_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\filter_channel_FT8_healthy\theta_band_FT8_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\filter_channel_F3_healthy\theta_band_F3_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\filter_channel_F4_healthy\theta_band_F4_healthy"
    ]
]

for channel_group in channel_groups:
    all_epochs_data = []  # This will hold the data for all epochs across all channels
    ch_names = ['FT7', 'T7', 'TP7', 'TP8', 'T8', 'FT8', 'F3', 'F4']
    info = None  # This will be used to store the info structure for creating combined epochs later
    for channel_dir in channel_group:
        channel_epochs_data = []  # This will hold data for all epochs for the current channel
        try:
            file_paths = find_fif_files(channel_dir)
            if not file_paths:
                print(f"No .fif files found in {channel_dir}. Skipping this directory.")
                continue
            for file_path in file_paths:
                try:
                    epochs = mne.read_epochs(file_path, preload=True)
                    print(f"{file_path} contains {len(epochs)} epochs")
                    if info is None:
                        info = epochs.info  # We only need to set this once
                    channel_epochs_data.append(epochs.get_data())  # Append data for these epochs
                except Exception as e:
                    print(f"Failed to read epochs from {file_path}. Error: {e}")
                    continue
        except Exception as e:
            print(f"Error processing directory {channel_dir}: {e}")
            continue
        if channel_epochs_data:
            # Concatenate all epoch data for the current channel
            channel_basename = os.path.basename(channel_dir)
            total_epochs = sum(len(e) for e in channel_epochs_data)
            print(f"Channel {channel_basename} combined total epochs: {total_epochs}")
            channel_combined_data = np.concatenate(channel_epochs_data, axis=0)
            all_epochs_data.append(channel_combined_data)

    if all_epochs_data:
        # Combine data from all channels, adjusting axis as necessary
        data_combined = np.concatenate(all_epochs_data, axis=1)  # Adjust axis based on your data structure
        print(f"Total combined epochs across all channels: {data_combined.shape[0]}")
        print(f"Diagnostic: Combined data shape: {data_combined.shape}")
        
    else:
        print("No epochs were loaded. Please check file paths and formats.")
        continue

    # Now, create the combined epochs structure
    if info is not None and data_combined.size > 0:
        # mapped_ch_names = [electrode_name_mapping.get(ch_name, ch_name) for ch_name in ch_names]
        # Update info to match the combined data
        info = mne.create_info(ch_names=ch_names, sfreq=info['sfreq'], ch_types=['eeg'] * len(ch_names))
        # Create a dummy events array and event_id dict, as they are required for creating EpochsArray
        events = np.array([[i, 0, 1] for i in range(data_combined.shape[0])])
        event_id = {'dummy_event': 1}
        epochs_combined = mne.EpochsArray(data_combined, info, events=events, event_id=event_id)
        print(f"Starting connectivity analysis on combined data...")
        processed_epochs_count = 0
        save_path = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\coherence_visualization_healthy\theta_band_coherence_healthy"
        save_csv_path = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\csv_data_coherence_healthy\theta_band_csv_healthy"
        for epoch_idx in range(len(epochs_combined)):
            # Here you can proceed with your connectivity analysis on `epochs_combined`
            con = spectral_connectivity_epochs(
                epochs_combined[epoch_idx:epoch_idx+1], method='coh', mode='multitaper', sfreq=epochs_combined.info['sfreq'],
                fmin=4, fmax=8, faverage=True, mt_adaptive=True, n_jobs=1
            )
            con_matrix = con.get_data(output='dense')[..., 0]
            df_con = pd.DataFrame(con_matrix, index=ch_names, columns=ch_names)
            processed_epochs_count += 1

            csv_file_name = f"coherence_matrix_Healthy_thetaBand_epoch_{epoch_idx + 1}.csv"
            csv_full_path = os.path.join(save_csv_path, csv_file_name)
            df_con.to_csv(csv_full_path)

            plt.figure(figsize=(10, 8))
            sns.heatmap(df_con, annot=True, cmap='viridis', fmt=".2f")
            plt.title('Coherence Matrix for Theta Band Healthy')
            plt.ylabel('Channels')
            plt.xlabel('Channels')
            file_name = f"coherence_matrix_Healthy_thetaBand_epoch_{epoch_idx + 1}.png"
            full_path = os.path.join(save_path, file_name)
            plt.savefig(full_path)
            plt.close()
            print(f"Processed and visualized {processed_epochs_count} epochs so far.")
            print(f"Total epochs processed for visualization: {processed_epochs_count}")
        else:
            print("No combined data available for creating epochs.")

## Theta band (dataset real)

In [ ]:
# electrode_name_mapping = {
#     'EEG039': 'FT7',
#     'EEG045': 'T7',
#     'EEG050': 'TP7',
#     'EEG101': 'TP8',
#     'EEG108': 'T8',
#     'EEG115': 'FT8',
#     'EEG024': 'F3',
#     'EEG124': 'F4'
# }

def find_edf_files(directory):
    return [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.edf')]

channel_groups = [
    [
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\filter_channel_C3-R_healthy\theta_band_C3-R_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\filter_channel_C4-R_healthy\theta_band_C4-R_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\filter_channel_FP1-R_healthy\theta_band_FP1-R_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\filter_channel_FP2-R_healthy\theta_band_FP2-R_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\filter_channel_O1-R_healthy\theta_band_O1-R_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\filter_channel_O2-R_healthy\theta_band_O2-R_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\filter_channel_T3-R_healthy\theta_band_T3-R_healthy",
        r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\filter_channel_T4-R_healthy\theta_band_T4-R_healthy"
    ]
]

for channel_group in channel_groups:
    all_epochs_data = []  # This will hold the data for all epochs across all channels
    ch_names = ['EEG FP1-R', 'EEG FP2-R', 'EEG C3-R', 'EEG C4-R', 'EEG O1-R', 'EEG O2-R', 'EEG T3-R', 'EEG T4-R']
    info = None  # This will be used to store the info structure for creating combined epochs later
    for channel_dir in channel_group:
        channel_epochs_data = []  # This will hold data for all epochs for the current channel
        try:
            file_paths = find_edf_files(channel_dir)
            if not file_paths:
                print(f"No .edf files found in {channel_dir}. Skipping this directory.")
                continue
            for file_path in file_paths:
                try:
                    epochs = mne.read_epochs(file_path, preload=True)
                    print(f"{file_path} contains {len(epochs)} epochs")
                    if info is None:
                        info = epochs.info  # We only need to set this once
                    channel_epochs_data.append(epochs.get_data())  # Append data for these epochs
                except Exception as e:
                    print(f"Failed to read epochs from {file_path}. Error: {e}")
                    continue
        except Exception as e:
            print(f"Error processing directory {channel_dir}: {e}")
            continue
        if channel_epochs_data:
            # Concatenate all epoch data for the current channel
            channel_basename = os.path.basename(channel_dir)
            total_epochs = sum(len(e) for e in channel_epochs_data)
            print(f"Channel {channel_basename} combined total epochs: {total_epochs}")
            channel_combined_data = np.concatenate(channel_epochs_data, axis=0)
            all_epochs_data.append(channel_combined_data)

    if all_epochs_data:
        # Combine data from all channels, adjusting axis as necessary
        data_combined = np.concatenate(all_epochs_data, axis=1)  # Adjust axis based on your data structure
        print(f"Total combined epochs across all channels: {data_combined.shape[0]}")
        print(f"Diagnostic: Combined data shape: {data_combined.shape}")
        
    else:
        print("No epochs were loaded. Please check file paths and formats.")
        continue

    # Now, create the combined epochs structure
    if info is not None and data_combined.size > 0:
        # mapped_ch_names = [electrode_name_mapping.get(ch_name, ch_name) for ch_name in ch_names]
        # Update info to match the combined data
        info = mne.create_info(ch_names=ch_names, sfreq=info['sfreq'], ch_types=['eeg'] * len(ch_names))
        # Create a dummy events array and event_id dict, as they are required for creating EpochsArray
        events = np.array([[i, 0, 1] for i in range(data_combined.shape[0])])
        event_id = {'dummy_event': 1}
        epochs_combined = mne.EpochsArray(data_combined, info, events=events, event_id=event_id)
        print(f"Starting connectivity analysis on combined data...")
        processed_epochs_count = 0
        save_path = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\coherence_visualization_healthy\theta_band_coherence_healthy"
        save_csv_path = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\csv_data_coherence_healthy\theta_band_csv_healthy"
        for epoch_idx in range(len(epochs_combined)):
            # Here you can proceed with your connectivity analysis on `epochs_combined`
            con = spectral_connectivity_epochs(
                epochs_combined[epoch_idx:epoch_idx+1], method='coh', mode='multitaper', sfreq=epochs_combined.info['sfreq'],
                fmin=4, fmax=8, faverage=True, mt_adaptive=True, n_jobs=1
            )
            con_matrix = con.get_data(output='dense')[..., 0]
            df_con = pd.DataFrame(con_matrix, index=ch_names, columns=ch_names)
            processed_epochs_count += 1

            csv_file_name = f"coherence_matrix_Healthy_thetaBand_epoch_{epoch_idx + 1}.csv"
            csv_full_path = os.path.join(save_csv_path, csv_file_name)
            df_con.to_csv(csv_full_path)

            plt.figure(figsize=(10, 8))
            sns.heatmap(df_con, annot=True, cmap='viridis', fmt=".2f")
            plt.title('Coherence Matrix for Theta Band Healthy')
            plt.ylabel('Channels')
            plt.xlabel('Channels')
            file_name = f"coherence_matrix_Healthy_thetaBand_epoch_{epoch_idx + 1}.png"
            full_path = os.path.join(save_path, file_name)
            plt.savefig(full_path)
            plt.close()
            print(f"Processed and visualized {processed_epochs_count} epochs so far.")
            print(f"Total epochs processed for visualization: {processed_epochs_count}")
        else:
            print("No combined data available for creating epochs.")

# Correlation on Healthy Dataset

In [ ]:
# Metode Korelasi Pearson dataset healthy
# electrode_name_mapping = {
#     'EEG039': 'FT7',
#     'EEG115': 'FT8',
#     'EEG045': 'T7',
#     'EEG108': 'T8',
#     'EEG050': 'TP7',
#     'EEG101': 'TP8',
#     'EEG024': 'F3',
#     'EEG124': 'F4'
# }

directory_path = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\6s_epochs_healthy"
selected_electrodes_names = ['FT7', 'T7', 'TP7', 'TP8', 'T8', 'FT8', 'F3', 'F4']
file_paths = [os.path.join(directory_path, file) for file in os.listdir(directory_path) if file.endswith(".fif")]
save_directory = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\correlation_visualization_healthy"
save_csv_path = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_2\Sumber_data\Healthy\csv_data_correlation_healthy"
os.makedirs(save_directory, exist_ok=True)
total_epochs = 0
for file_path in file_paths:
    epochs = mne.read_epochs(file_path, preload=True)
    num_epochs = len(epochs)
    total_epochs += num_epochs
    file_basename = os.path.basename(file_path).split('.')[0]
    print(f"{os.path.basename(file_path)}: {num_epochs} epochs")

    for epoch_idx, epoch_data in enumerate(epochs.get_data()):
        selected_indices = [epochs.ch_names.index(name) for name in selected_electrodes_names if name in epochs.ch_names]
        selected_data = epoch_data[selected_indices]
        column_labels = [epochs.ch_names[idx] for idx in selected_indices]
        df = pd.DataFrame(selected_data.T, columns=column_labels)
        correlation_matrix = df.corr()
        
        csv_filename = f"{file_basename}_correlation_matrix_epoch_{epoch_idx + 1}.csv"
        csv_file_path = os.path.join(save_csv_path, csv_filename)
        correlation_matrix.to_csv(csv_file_path)

        plt.figure(figsize=(10, 8))
        sns.heatmap(correlation_matrix, annot=True, cmap="viridis", fmt=".2f", 
                    xticklabels=column_labels, 
                    yticklabels=column_labels)
        file_name = f"{file_basename}_correlation_matrix_epoch_{epoch_idx + 1}.png"
        plt.title(f"Pearson Correlation Matrix")
        save_path = os.path.join(save_directory, file_name) 
        plt.savefig(save_path)
        plt.close()
print(f"Total epochs loaded: {total_epochs}")

# Correlation on healthy dataset real

In [ ]:
# Metode Korelasi Pearson dataset healthy
# electrode_name_mapping = {
#     'EEG039': 'FT7',
#     'EEG115': 'FT8',
#     'EEG045': 'T7',
#     'EEG108': 'T8',
#     'EEG050': 'TP7',
#     'EEG101': 'TP8',
#     'EEG024': 'F3',
#     'EEG124': 'F4'
# }

directory_path = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\6s_epochs_healthy"
selected_electrodes_names = ['EEG FP1-R', 'EEG FP2-R', 'EEG C3-R', 'EEG C4-R', 'EEG O1-R', 'EEG O2-R', 'EEG T3-R', 'EEG T4-R']
file_paths = [os.path.join(directory_path, file) for file in os.listdir(directory_path) if file.endswith(".edf")]
save_directory = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\correlation_visualization_healthy"
save_csv_path = r"E:\Vscode\Tugas Anime\Workspace\Eksperimen_4\Healthy\csv_data_correlation_healthy"
os.makedirs(save_directory, exist_ok=True)
total_epochs = 0
for file_path in file_paths:
    epochs = mne.read_epochs(file_path, preload=True)
    num_epochs = len(epochs)
    total_epochs += num_epochs
    file_basename = os.path.basename(file_path).split('.')[0]
    print(f"{os.path.basename(file_path)}: {num_epochs} epochs")

    for epoch_idx, epoch_data in enumerate(epochs.get_data()):
        selected_indices = [epochs.ch_names.index(name) for name in selected_electrodes_names if name in epochs.ch_names]
        selected_data = epoch_data[selected_indices]
        column_labels = [epochs.ch_names[idx] for idx in selected_indices]
        df = pd.DataFrame(selected_data.T, columns=column_labels)
        correlation_matrix = df.corr()
        
        csv_filename = f"{file_basename}_correlation_matrix_epoch_{epoch_idx + 1}.csv"
        csv_file_path = os.path.join(save_csv_path, csv_filename)
        correlation_matrix.to_csv(csv_file_path)

        plt.figure(figsize=(10, 8))
        sns.heatmap(correlation_matrix, annot=True, cmap="viridis", fmt=".2f", 
                    xticklabels=column_labels, 
                    yticklabels=column_labels)
        file_name = f"{file_basename}_correlation_matrix_epoch_{epoch_idx + 1}.png"
        plt.title(f"Pearson Correlation Matrix")
        save_path = os.path.join(save_directory, file_name) 
        plt.savefig(save_path)
        plt.close()
print(f"Total epochs loaded: {total_epochs}")